In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_scoren
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data.drop(['Adj Close', 'Volume'], axis=1)

def prepare_data_separate(train_data_list, valid_data_list, seq_len, target_col, symbol, start_date, end_date, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        train_data = train_data.copy()

        # Create separate dataframes for prices and volume
        train_data_reshaped = train_data.values.reshape(-1, 1)

        train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # Reshape it back to original shape.
        train_data[train_data.columns] = train_data_transformed.reshape(-1, 4)

        # Shift target column by forward steps.
        train_data['Target'] = train_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        train_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        # train_data = train_data.copy()
        # train_data['Target'] = train_data[target_col_name].shift(forward)
        # train_data.dropna(inplace=True)
        # train_data = train_data.drop(target_col_name, axis=1)
        
        # train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        valid_data = valid_data.copy()

        # Create separate dataframes for prices and volume
        valid_data_reshaped = valid_data.values.reshape(-1, 1)

        valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # Reshape it back to original shape.
        valid_data[valid_data.columns] = valid_data_transformed.reshape(-1, 4)

        # Shift target column by forward steps.
        valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        valid_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        valid_data.drop(target_col_name, axis=1, inplace=True)
        
        # valid_data = valid_data.copy()
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)
        # valid_data.dropna(inplace=True)
        # valid_data = valid_data.drop(target_col_name, axis=1)

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
  
    test_data = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_reshaped = test_data.values.reshape(-1, 1)

    test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # Reshape it back to original shape.
    test_data[test_data.columns] = test_data_transformed.reshape(-1, 4)

    # Shift target column by forward steps.
    test_data['Target'] = test_data[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data.drop(target_col_name, axis=1, inplace=True)
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, seq_len
    
def prepare_data_common(train_data, valid_data, test_data, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)
    X_test, y_test = create_sequences(test_data, seq_len)
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)


    return X_train, y_train, X_valid, y_valid, X_test, y_test

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-1,:])
        output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]

        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, n_predict, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)
    
    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
        
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, 0, -1], label='Actual')
        plt.plot(time_steps, output_org[:, 0, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-n_predict:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, 0, j], label='Actual')
            ax.plot(time_steps, output_org[:, 0, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-n_predict:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2
          
def predict_future(model, X_test, n_predict, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
        return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
        future_predictions = []

        for i in range(n_predict):
            # Generate a prediction
            recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
            with torch.no_grad():
                input_seq = torch.Tensor(recent_input_sequence).to(device)
                output = model(input_seq).cpu().numpy() 

                future_prediction = output[0, 0, :]

            # Append the prediction to the future_predictions list
            future_predictions.append(future_prediction)         
     
            # Update the input sequence with the new prediction, if not the last iteration
            if i < n_predict - 1:
                recent_input_sequence = update_sequence(recent_input_sequence, future_prediction, sequence_length)

            else:
                break
        
        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
    return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, valid_size=0.5, 
                  n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # Generate random hyperparameters and parameters
        seq_len = random.choice(range(5, 6))
        nlayers = random.choice(range(1, 2))
        nneurons = random.choice(range(32, 33))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(300, 1000))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001])
        patience = random.choice(range(20, 21))
        min_delta = random.choice([0.00005])
        l2_regularization = random.choice([0])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, seq_len=seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, seq_len= prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date,
                                                                                        seq_len=seq_len, target_col=target_col, forward=forward)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data, seq_len=seq_len)
        
        input_shape = (X_train.shape[0], seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # Inverse transform the y_test to the original scale
        test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_predict > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test, n_predict=n_predict,
                                                        n_last_sequence=n_last_sequence, scaler=test_scaler)
        # print(f"Future Predictions (Trial {trial+1}): {future_predictions.shape}")
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(n_predict*n_predict):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label,
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > n_predict:
                    y_true_all_features = y_test_org[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                # else:
                #     y_true_all_features = y_test_org[-(n_last_sequence-i):, -1]
                #     y_true = y_true_all_features[:, -1]
                #     y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_pred = y_pred_all_features[:, -1]

                #     # Inverse transform the y_test to the original scale
                #     y_actual_all_features = y_test[-(n_last_sequence-i):, -1]
                #     y_actual = y_actual_all_features[:, -1]
                #     y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# modified_short test data 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data.drop(['Adj Close', 'Volume'], axis=1)

def prepare_data_separate(train_data_list, valid_data_list, seq_len, target_col, symbol, start_date, end_date, 
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        train_data = train_data.copy()

        # Create separate dataframes for prices and volume
        train_data_reshaped = train_data.values.reshape(-1, 1)

        train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # Reshape it back to original shape.
        train_data[train_data.columns] = train_data_transformed.reshape(train_data.shape)

        # Shift target column by forward steps.
        train_data['Target'] = train_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        train_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        # train_data = train_data.copy()
        # train_data['Target'] = train_data[target_col_name].shift(forward)
        # train_data.dropna(inplace=True)
        # train_data = train_data.drop(target_col_name, axis=1)
        
        # train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        valid_data = valid_data.copy()

        # Create separate dataframes for prices and volume
        valid_data_reshaped = valid_data.values.reshape(-1, 1)

        valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # Reshape it back to original shape.
        valid_data[valid_data.columns] = valid_data_transformed.reshape(valid_data.shape)

        # Shift target column by forward steps.
        valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        valid_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        valid_data.drop(target_col_name, axis=1, inplace=True)
        
        # valid_data = valid_data.copy()
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)
        # valid_data.dropna(inplace=True)
        # valid_data = valid_data.drop(target_col_name, axis=1)

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
  
    test_data = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_reshaped = test_data.values.reshape(-1, 1)

    test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # Reshape it back to original shape.
    test_data[test_data.columns] = test_data_transformed.reshape(test_data.shape)

    # Shift target column by forward steps.
    test_data['Target'] = test_data[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data.drop(target_col_name, axis=1, inplace=True)
    
    
    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)
    
    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)    

    test_data_short = test_data_short.copy()

    # Create separate dataframes for prices and volume
    test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # Reshape it back to original shape.
    test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(test_data_short.shape)

    # Shift target column by forward steps.
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data_short.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data_short.drop(target_col_name, axis=1, inplace=True)    
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, seq_len
    
def prepare_data_common(train_data, valid_data, test_data, test_data_short, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)
    X_test, y_test = create_sequences(test_data, seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, seq_len)
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-1,:])
        output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]

        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, n_predict, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)
    
    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
        
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, 0, -1], label='Actual')
        plt.plot(time_steps, output_org[:, 0, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-n_predict:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1}_{test_length})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, 0, j], label='Actual')
            ax.plot(time_steps, output_org[:, 0, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-n_predict:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1}_{test_length})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2
          
def predict_future(model, X_test, n_predict, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
        return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
        future_predictions = []

        for i in range(n_predict):
            # Generate a prediction
            recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
            with torch.no_grad():
                input_seq = torch.Tensor(recent_input_sequence).to(device)
                output = model(input_seq).cpu().numpy() 

                future_prediction = output[0, 0, :]

            # Append the prediction to the future_predictions list
            future_predictions.append(future_prediction)         
     
            # Update the input sequence with the new prediction, if not the last iteration
            if i < n_predict - 1:
                recent_input_sequence = update_sequence(recent_input_sequence, future_prediction, sequence_length)

            else:
                break
        
        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
    return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, start_date_short=None, end_date_short=None,
                  valid_size=0.5, n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # # Generate random hyperparameters and parameters
        # seq_len = random.choice(range(5, 6))
        # nlayers = random.choice(range(1, 2))
        # nneurons = random.choice(range(32, 33))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # dropout = random.choice([0])
        # optimizer = random.choice([torch.optim.Adam])
        # n_epochs = random.choice(range(300, 1000))
        # batch_size = random.choice(range(256, 512))
        # learning_rate = random.choice([0.0001])
        # patience = random.choice(range(20, 21))
        # min_delta = random.choice([0.00005])
        # l2_regularization = random.choice([0])
        
        # Generate random hyperparameters and parameters
        seq_len = random.choice(range(5, 31))
        nlayers = random.choice(range(1, 6))
        nneurons = random.choice(range(16, 128))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0.1, 0.2, 0.3, 0.4])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(300, 1000))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001, 0.0005, 0.001])
        patience = random.choice(range(5, 21))
        min_delta = random.choice([0.0001, 0.0002])
        l2_regularization = random.choice([0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, seq_len=seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, seq_len= prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short, 
                                                                                        end_date_short=end_date_short, seq_len=seq_len, target_col=target_col, forward=forward)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, 
                                                                                                                 test_data=test_data, test_data_short=test_data_short, seq_len=seq_len)
        
        input_shape = (X_train.shape[0], seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # Inverse transform the y_test to the original scale
        test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        
        test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)
        
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_predict > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test, n_predict=n_predict,
                                                        n_last_sequence=n_last_sequence, scaler=test_scaler)
        # print(f"Future Predictions (Trial {trial+1}): {future_predictions.shape}")
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(n_predict*n_predict):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label, test_length="long",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)
            
            plot_predictions(loaded_model, X_test_short, y_test_short, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler_short, col_label=col_label, test_length="short",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > n_predict:
                    y_true_all_features = y_test_org[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                # else:
                #     y_true_all_features = y_test_org[-(n_last_sequence-i):, -1]
                #     y_true = y_true_all_features[:, -1]
                #     y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_pred = y_pred_all_features[:, -1]

                #     # Inverse transform the y_test to the original scale
                #     y_actual_all_features = y_test[-(n_last_sequence-i):, -1]
                #     y_actual = y_actual_all_features[:, -1]
                #     y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# modified_2_short test data_+data_shift
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data.drop(['Adj Close', 'Volume'], axis=1)

def prepare_data_separate(train_data_list, valid_data_list, seq_len, target_col, symbol, start_date, end_date, 
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        train_data = train_data.copy()

        # # Create separate dataframes for prices and volume
        # train_data_reshaped = train_data.values.reshape(-1, 1)

        # train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # # Reshape it back to original shape.
        # train_data[train_data.columns] = train_data_transformed.reshape(train_data.shape)

        # # Shift target column by forward steps.
        # train_data['Target'] = train_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # train_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        train_data = train_data.copy()
        train_data['Target'] = train_data[target_col_name].shift(forward)
        train_data.dropna(inplace=True)
        train_data = train_data.drop(target_col_name, axis=1)
        
        train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        # valid_data = valid_data.copy()

        # # Create separate dataframes for prices and volume
        # valid_data_reshaped = valid_data.values.reshape(-1, 1)

        # valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # # Reshape it back to original shape.
        # valid_data[valid_data.columns] = valid_data_transformed.reshape(valid_data.shape)

        # # Shift target column by forward steps.
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # valid_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # valid_data.drop(target_col_name, axis=1, inplace=True)
        
        valid_data = valid_data.copy()
        valid_data['Target'] = valid_data[target_col_name].shift(forward)
        valid_data.dropna(inplace=True)
        valid_data = valid_data.drop(target_col_name, axis=1)

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
    
    test_data_unshifted = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_unshifted_reshaped = test_data_unshifted.values.reshape(-1, 1)

    test_data_unshifted_transformed = scaler.fit_transform(test_data_unshifted_reshaped)

    # Reshape it back to original shape.
    test_data_unshifted[test_data_unshifted.columns] = test_data_unshifted_transformed.reshape(test_data_unshifted.shape)

    # Shift target column by forward steps.
    test_data_unshifted['Target'] = test_data_unshifted[target_col_name]

    # Drop NA values if there are any due to shifting.
    test_data_unshifted.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data_unshifted.drop(target_col_name, axis=1, inplace=True)
 
    test_data = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_reshaped = test_data.values.reshape(-1, 1)

    test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # Reshape it back to original shape.
    test_data[test_data.columns] = test_data_transformed.reshape(test_data.shape)

    # Shift target column by forward steps.
    test_data['Target'] = test_data[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data.drop(target_col_name, axis=1, inplace=True)
    
    
    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)
    
    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)    

    test_data_short = test_data_short.copy()

    # Create separate dataframes for prices and volume
    test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # Reshape it back to original shape.
    test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(test_data_short.shape)

    # Shift target column by forward steps.
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data_short.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data_short.drop(target_col_name, axis=1, inplace=True)    
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_unshifted, test_data_short, test_data_short_unnormalized, seq_len
    
def prepare_data_common(train_data, valid_data, test_data, test_data_unshifted, test_data_short, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)
    X_test, y_test = create_sequences(test_data, seq_len)
    X_test_unshifted, y_test_unshifted = create_sequences(test_data_unshifted, seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, seq_len)
    
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_unshifted = torch.Tensor(X_test_unshifted)
    y_test_unshifted = torch.Tensor(y_test_unshifted)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_unshifted, y_test_unshifted, X_test_short, y_test_short

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-1,:])
        output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]

        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, n_predict, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)
    
    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
        
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, 0, -1], label='Actual')
        plt.plot(time_steps, output_org[:, 0, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-n_predict:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1}_{test_length})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, 0, j], label='Actual')
            ax.plot(time_steps, output_org[:, 0, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-n_predict:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1}_{test_length})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2
          
# def predict_future(model, X_test, y_test_unshifted, n_predict=5, forward=-1, n_last_sequence=1, scaler=None):
#     n_features = X_test.shape[2]
#     sequence_length = X_test.shape[1]
#     torch.cuda.empty_cache()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
#     model.eval()

#     def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
#         return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
#     # def new_sequence(last_sequences, y_test, sequence_length):
#     #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
#     # Prepare the most recent input sequence
#     # x_test_sequences = X_test[-(n_last_sequence):, :, :]
#     # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
#     # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
#     y_test_unshifted_sequences = y_test_unshifted[-(n_last_sequence+forward):, :, :]
    
#     last_sequences = X_test[-(n_last_sequence):, :, :]
#     last_sequences = torch.Tensor(last_sequences)
    
#     merge_future_predictions = None

#     for idx, recent_input_sequence in enumerate(last_sequences):

#         future_predictions = []

#         for i in range(n_predict):
#             # Generate a prediction
#             recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
#             with torch.no_grad():
#                 input_seq = torch.Tensor(recent_input_sequence).to(device)
#                 output = model(input_seq).cpu().numpy() 
#                 # Use only the last feature from output and substitute other features with those from recent_input_sequence
#                 last_feature_prediction = output[0, 0, -1:]  # Shape should be (1,)
                
#                 future_prediction = output[0, 0, :]

#             # Append the prediction to the future_predictions list
#             future_predictions.append(future_prediction)         
     
#             # Update the input sequence with the new prediction, if not the last iteration
#             if idx + i + 1 < len(y_test_unshifted_sequences):
                
#                 other_features_from_y_test_unshifted = y_test_unshifted_sequences[idx+i+1, -1:, :-1]  # Shape should be (n_features-1,)
#                 future_next_prediction = np.concatenate([other_features_from_y_test_unshifted.flatten(), last_feature_prediction])
                
#                 recent_input_sequence = update_sequence(recent_input_sequence, future_next_prediction, sequence_length)
#             else:
#                 break
        
#         future_predictions_array = np.array(future_predictions)
#         future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

#         if merge_future_predictions is None:
#             merge_future_predictions = future_predictions_inverse
#             merge_future_predictions_org = future_predictions_array
#         else:
#             merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
#             merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
#     return merge_future_predictions, merge_future_predictions_org

def predict_future(model, X_test, n_predict, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
        return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
        future_predictions = []

        for i in range(n_predict):
            # Generate a prediction
            recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
            with torch.no_grad():
                input_seq = torch.Tensor(recent_input_sequence).to(device)
                output = model(input_seq).cpu().numpy() 

                future_prediction = output[0, 0, :]

            # Append the prediction to the future_predictions list
            future_predictions.append(future_prediction)         
     
            # Update the input sequence with the new prediction, if not the last iteration
            if i < n_predict - 1:
                recent_input_sequence = update_sequence(recent_input_sequence, future_prediction, sequence_length)

            else:
                break
        
        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
    return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, start_date_short=None, end_date_short=None,
                  valid_size=0.5, n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # Generate random hyperparameters and parameters
        seq_len = random.choice(range(5, 6))
        nlayers = random.choice(range(1, 2))
        nneurons = random.choice(range(32, 33))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(500, 1000))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001])
        patience = random.choice(range(50, 51))
        min_delta = random.choice([0.00005])
        l2_regularization = random.choice([0])
        
        # # Generate random hyperparameters and parameters
        # seq_len = random.choice(range(10, 11))
        # nlayers = random.choice(range(1, 2))
        # nneurons = random.choice(range(3000, 3001))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # dropout = random.choice([0.1, 0.2, 0.3, 0.4])
        # optimizer = random.choice([torch.optim.Adam])
        # n_epochs = random.choice(range(200,201))
        # batch_size = random.choice(range(256, 257))
        # learning_rate = random.choice([0.0001, 0.0005, 0.001])
        # patience = random.choice(range(20, 21))
        # min_delta = random.choice([0.0001, 0.0002])
        # l2_regularization = random.choice([0.01, 0.05, 0.1])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, seq_len=seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_unshifted, test_data_short, test_data_short_unnormalized, seq_len= prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short, 
                                                                                        end_date_short=end_date_short, seq_len=seq_len, target_col=target_col, forward=forward)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_unshifted, y_test_unshifted, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data, 
                                                                                                                                                     test_data_unshifted=test_data_unshifted, test_data_short=test_data_short, seq_len=seq_len)
        
        input_shape = (X_train.shape[0], seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # Inverse transform the y_test to the original scale
        test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        
        test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)
        
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_predict > 0:
            # future_predictions, future_predictions_org = predict_future(loaded_model, X_test, y_test_unshifted, n_predict=n_predict,
            #                                             forward=forward, n_last_sequence=n_last_sequence, scaler=test_scaler)
                future_predictions, future_predictions_org = predict_future(loaded_model, X_test, n_predict=n_predict,
                                                n_last_sequence=n_last_sequence, scaler=test_scaler)
        # print(f"Future Predictions (Trial {trial+1}): {future_predictions.shape}")
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(n_predict):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label, test_length="long",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)
            
            plot_predictions(loaded_model, X_test_short, y_test_short, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler_short, col_label=col_label, test_length="short",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > n_predict:
                    y_true_all_features = y_test_org[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                else:
                    y_true_all_features = y_test_org[-(n_last_sequence-i):, -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence-i):, -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                    y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# modified_2_short test data_+data_shift
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data.drop(['Adj Close', 'Volume', 'Open', 'High', 'Low'], axis=1)
    # return data.drop('Adj Close', axis=1)

def prepare_data_separate(train_data_list, valid_data_list, seq_len, target_col, symbol, start_date, end_date, 
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        train_data = train_data.copy()

        # Create separate dataframes for prices and volume
        train_data_reshaped = train_data.values.reshape(-1, 1)

        train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # Reshape it back to original shape.
        train_data[train_data.columns] = train_data_transformed.reshape(train_data.shape)

        # Shift target column by forward steps.
        train_data['Target'] = train_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        train_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        # train_data = train_data.copy()
        # train_data['Target'] = train_data[target_col_name].shift(forward)
        # train_data.dropna(inplace=True)
        # train_data = train_data.drop(target_col_name, axis=1)
        
        # train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        valid_data = valid_data.copy()

        # Create separate dataframes for prices and volume
        valid_data_reshaped = valid_data.values.reshape(-1, 1)

        valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # Reshape it back to original shape.
        valid_data[valid_data.columns] = valid_data_transformed.reshape(valid_data.shape)

        # Shift target column by forward steps.
        valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # Drop NA values if there are any due to shifting.
        valid_data.dropna(inplace=True)

        # Drop original target column after creating shifted Target.
        valid_data.drop(target_col_name, axis=1, inplace=True)
        
        # valid_data = valid_data.copy()
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)
        # valid_data.dropna(inplace=True)
        # valid_data = valid_data.drop(target_col_name, axis=1)

        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
    
    test_data_unshifted = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_unshifted_reshaped = test_data_unshifted.values.reshape(-1, 1)

    test_data_unshifted_transformed = scaler.fit_transform(test_data_unshifted_reshaped)

    # Reshape it back to original shape.
    test_data_unshifted[test_data_unshifted.columns] = test_data_unshifted_transformed.reshape(test_data_unshifted.shape)

    # Shift target column by forward steps.
    test_data_unshifted['Target'] = test_data_unshifted[target_col_name]

    # Drop NA values if there are any due to shifting.
    test_data_unshifted.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data_unshifted.drop(target_col_name, axis=1, inplace=True)
 
    test_data = test_data.copy()

    # Create separate dataframes for prices and volume
    test_data_reshaped = test_data.values.reshape(-1, 1)

    test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # Reshape it back to original shape.
    test_data[test_data.columns] = test_data_transformed.reshape(test_data.shape)

    # Shift target column by forward steps.
    test_data['Target'] = test_data[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data.drop(target_col_name, axis=1, inplace=True)
    
    # # Fetch a fresh copy of the test data
    # test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # # Scale test data
    # test_data_unnormalized = test_data.copy()
    # test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    # test_data_unnormalized.dropna(inplace=True)
    # test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)

    # test_data_unshifted = test_data.copy()
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]
    # test_data_unshifted.dropna(inplace=True)
    # test_data_unshifted = test_data_unshifted.drop(target_col_name, axis=1)
    # test_data_unshifted[test_data_unshifted.columns] = scaler.fit_transform(test_data_unshifted)

    # # Scale test data
    # test_data = test_data.copy()
    # test_data[test_data.columns] = scaler.fit_transform(test_data)
    # test_data['Target'] = test_data[target_col_name].shift(forward)
    # test_data.dropna(inplace=True)
    # test_data = test_data.drop(target_col_name, axis=1)    
    
    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)
    
    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)    

    test_data_short = test_data_short.copy()

    # Create separate dataframes for prices and volume
    test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # Reshape it back to original shape.
    test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(test_data_short.shape)

    # Shift target column by forward steps.
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # Drop NA values if there are any due to shifting.
    test_data_short.dropna(inplace=True)

    # Drop original target column after creating shifted Target.
    test_data_short.drop(target_col_name, axis=1, inplace=True)
    
    # # Scale test data
    # test_data_short = test_data_short.copy()
    # test_data_short[test_data_short.columns] = scaler.fit_transform(test_data_short)
    # test_data_short['Target'] = test_data_short[target_col_name].shift(forward)
    # test_data_short.dropna(inplace=True)
    # test_data_short = test_data_short.drop(target_col_name, axis=1) 
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_unshifted, test_data_short, test_data_short_unnormalized, seq_len
    
def prepare_data_common(train_data, valid_data, test_data, test_data_unshifted, test_data_short, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)
    X_test, y_test = create_sequences(test_data, seq_len)
    X_test_unshifted, y_test_unshifted = create_sequences(test_data_unshifted, seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, seq_len)
    
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_unshifted = torch.Tensor(X_test_unshifted)
    y_test_unshifted = torch.Tensor(y_test_unshifted)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)
    
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}, X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}, X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_unshifted, y_test_unshifted, X_test_short, y_test_short
   

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-1,:])
        output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()
        print(f"y_shape: {y.shape}, y_pred_shape: {y_pred.shape}\n")
        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]
        print(f"y_shape: {y.shape}, y_pred_shape: {y_pred.shape}\n")
        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, n_predict, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)
    
    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
        
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, 0, -1], label='Actual')
        plt.plot(time_steps, output_org[:, 0, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-n_predict:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1}_{test_length})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, 0, j], label='Actual')
            ax.plot(time_steps, output_org[:, 0, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-n_predict:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1}_{test_length})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2
          
def predict_future(model, X_test, n_predict, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
        return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
        future_predictions = []

        for i in range(n_predict):
            # Generate a prediction
            recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
            with torch.no_grad():
                input_seq = torch.Tensor(recent_input_sequence).to(device)
                output = model(input_seq).cpu().numpy() 

                future_prediction = output[0, 0, :]

            # Append the prediction to the future_predictions list
            future_predictions.append(future_prediction)         
     
            # Update the input sequence with the new prediction, if not the last iteration
            if i < n_predict - 1:
                recent_input_sequence = update_sequence(recent_input_sequence, future_prediction, sequence_length)

            else:
                break
        
        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
    return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, start_date_short=None, end_date_short=None,
                  valid_size=0.5, n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # Generate random hyperparameters and parameters
        seq_len = random.choice(range(10, 11))
        nlayers = random.choice(range(1, 2))
        nneurons = random.choice(range(100, 101))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(300, 500))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001, 0.0005])
        patience = random.choice(range(50, 51))
        min_delta = random.choice([0.0001])
        l2_regularization = random.choice([0])
        
        # # Generate random hyperparameters and parameters
        # seq_len = random.choice(range(20, 61))
        # nlayers = random.choice(range(1, 6))
        # nneurons = random.choice(range(100, 256))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # dropout = random.choice([0.1, 0.2, 0.3, 0.4])
        # optimizer = random.choice([torch.optim.Adam])
        # n_epochs = random.choice(range(300, 500))
        # batch_size = random.choice(range(128, 256))
        # learning_rate = random.choice([0.0001, 0.0005, 0.001])
        # patience = random.choice(range(5, 21))
        # min_delta = random.choice([0.0001, 0.0002])
        # l2_regularization = random.choice([0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, seq_len=seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_unshifted, test_data_short, test_data_short_unnormalized, seq_len= prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short, 
                                                                                        end_date_short=end_date_short, seq_len=seq_len, target_col=target_col, forward=forward)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_unshifted, y_test_unshifted, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data, 
                                                                                                                                                     test_data_unshifted=test_data_unshifted, test_data_short=test_data_short, seq_len=seq_len)
        
        input_shape = (X_train.shape[0], seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # Inverse transform the y_test to the original scale
        test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        
        test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)
        
        # # Inverse transform the y_test to the original scale 
        # test_scaler = StandardScaler().fit(test_data_unnormalized)
        
        # test_scaler_short = StandardScaler().fit(test_data_short_unnormalized)
        
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_predict > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test, n_predict=n_predict, 
                                                        n_last_sequence=n_last_sequence, scaler=test_scaler)
        # print(f"Future Predictions (Trial {trial+1}): {future_predictions.shape}")
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(n_predict*n_predict):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label, test_length="long",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)
            
            plot_predictions(loaded_model, X_test_short, y_test_short, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler_short, col_label=col_label, test_length="short",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > n_predict:
                    y_true_all_features = y_test_org[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                # else:
                #     y_true_all_features = y_test_org[-(n_last_sequence+1-i):, -1]
                #     y_true = y_true_all_features[:, -1]
                #     y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_pred = y_pred_all_features[:, -1]

                #     # Inverse transform the y_test to the original scale
                #     y_actual_all_features = y_test[-(n_last_sequence+1-i):, -1]
                #     y_actual = y_actual_all_features[:, -1]
                #     y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# modified_2_short test data_+data_shift
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


def create_sequences(data, seq_len):
    X = []
    y = []
    data = data.values  # This line is added
    for i in range(seq_len, data.shape[0]):
        X.append(data[i-seq_len:i, :])
        y.append(data[i:i+1, :])  # Change target shape to (1, n_features)

    X = np.array(X)
    y = np.array(y)
    return X, y

def create_differences(data):
    # Check if the required columns exist in the data
    if set(['Open', 'Close', 'High', 'Low']).issubset(data.columns):
        # Create new columns
        data['Diff_1'] = data['Close'] - data['Open']
        data['Diff_2'] = data['High'] - data['Low']

        # Drop unnecessary columns
        data = data.drop(columns=['Open', 'Low', 'High'])
        
    else:
        print("One or more of the required columns ('Open', 'Close', 'High', 'Low') are not present in the input dataframe.")
    
    return data

def calculate_indicators(data, rsi_period=14, short_ema_period=12, long_ema_period=26, signal_period=9, vol_period=20):
    # Check if 'Close' column exists in the data
    if 'Close' in data.columns:
            
        # Calculate 90 days moving average of 'Close'
        data['90D_MA_Close'] = data['Close'].rolling(window=90).mean()

        # Calculate 25 days moving average of the newly created '90D_MA_Close'
        data['25D_MA_of_90D'] = data['90D_MA_Close'].rolling(window=25).mean()
        
        # Calculate 20 days moving average of 'Close'
        data['25D_MA_Close'] = data['Close'].rolling(window=25).mean()
        
        # Calculate 14 days moving average of 'Close'
        data['14D_MA_Close'] = data['Close'].rolling(window=14).mean()
        
        # # Calculate daily returns        
        # data['Return'] = data['Close'].pct_change()

        # Calculate RSI
        delta = data['Close'].diff()
        up, down = delta.copy(), delta.copy()
        
        up[up < 0] = 0
        down[down > 0] = 0

        average_gain = up.rolling(window=rsi_period).mean()
        average_loss = abs(down.rolling(window=rsi_period).mean())

        rs = average_gain / average_loss

        data['RSI'] = 100 - (100 / (1 + rs))

        # Calculate MACD Line: (12-day EMA - 26-day EMA)
        EMA_short = data['Close'].ewm(span=short_ema_period).mean() 
        EMA_long = data['Close'].ewm(span=long_ema_period).mean() 
        data['MACD_Line'] = EMA_short - EMA_long

        # Calculate Signal Line: a n-day MA of MACD Line 
        data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

        # Calculate Volatility as rolling standard deviation of log returns
        data["Log_Return"] = np.log(data["Close"]).diff()
        data["Volatility"] = data["Log_Return"].rolling(window=vol_period).std()
        # Drop unnecessary columns
        data = data.drop(columns=['Open', 'Low', 'High', 'Log_Return'])  

    else:
         print("'Close' column is not present in the input dataframe.")
    
    return data

def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    # return data.drop(['Adj Close', 'Volume'], axis=1)
    return data.drop('Adj Close', axis=1)

def prepare_data_separate(train_data_list, valid_data_list, seq_len, target_col, symbol, start_date, end_date, 
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        # train_data = train_data.copy()

        # # Create separate dataframes for prices and volume
        # train_data_reshaped = train_data.values.reshape(-1, 1)

        # train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # # Reshape it back to original shape.
        # train_data[train_data.columns] = train_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # train_data['Target'] = train_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # train_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        train_data = train_data.copy()
        train_data = calculate_indicators(train_data)
        
        train_data['Target'] = train_data[target_col_name].shift(forward)
        train_data.dropna(inplace=True)
        train_data = train_data.drop(target_col_name, axis=1)
        
        train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        # valid_data = valid_data.copy()

        # # Create separate dataframes for prices and volume
        # valid_data_reshaped = valid_data.values.reshape(-1, 1)

        # valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # # Reshape it back to original shape.
        # valid_data[valid_data.columns] = valid_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # valid_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # valid_data.drop(target_col_name, axis=1, inplace=True)
        
        valid_data = valid_data.copy()
        valid_data = calculate_indicators(valid_data)
        
        valid_data['Target'] = valid_data[target_col_name].shift(forward)
        valid_data.dropna(inplace=True)
        valid_data = valid_data.drop(target_col_name, axis=1)
        valid_data[valid_data.columns] = scaler.fit_transform(valid_data)
        
        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
    
    # test_data_unshifted = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_unshifted_reshaped = test_data_unshifted.values.reshape(-1, 1)

    # test_data_unshifted_transformed = scaler.fit_transform(test_data_unshifted_reshaped)

    # # Reshape it back to original shape.
    # test_data_unshifted[test_data_unshifted.columns] = test_data_unshifted_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]

    # # Drop NA values if there are any due to shifting.
    # test_data_unshifted.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_unshifted.drop(target_col_name, axis=1, inplace=True)
 
    # test_data = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_reshaped = test_data.values.reshape(-1, 1)

    # test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # # Reshape it back to original shape.
    # test_data[test_data.columns] = test_data_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data['Target'] = test_data[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data.drop(target_col_name, axis=1, inplace=True)
    
    # # Fetch a fresh copy of the test data
    # test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)
    
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)

    # test_data_unshifted = test_data.copy()
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]
    # test_data_unshifted.dropna(inplace=True)
    # test_data_unshifted = test_data_unshifted.drop(target_col_name, axis=1)
    # test_data_unshifted[test_data_unshifted.columns] = scaler.fit_transform(test_data_unshifted)

    # Scale test data
    test_data = test_data.copy()
    test_data = calculate_indicators(test_data)
    
    test_data[test_data.columns] = scaler.fit_transform(test_data)
    test_data['Target'] = test_data[target_col_name].shift(forward)
    test_data.dropna(inplace=True)
    test_data = test_data.drop(target_col_name, axis=1)    
    
    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)
    
    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized = calculate_indicators(test_data_short_unnormalized)
    
    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)    

    # test_data_short = test_data_short.copy()

    # # Create separate dataframes for prices and volume
    # test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    # test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # # Reshape it back to original shape.
    # test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data_short.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_short.drop(target_col_name, axis=1, inplace=True)
    
    # Scale test data
    test_data_short = test_data_short.copy()
    test_data_short = calculate_indicators(test_data_short)
    test_data_short[test_data_short.columns] = scaler.fit_transform(test_data_short)
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)
    test_data_short.dropna(inplace=True)
    test_data_short = test_data_short.drop(target_col_name, axis=1) 
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, seq_len
    
def prepare_data_common(train_data, valid_data, test_data, test_data_short, seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, seq_len)
    X_valid, y_valid = create_sequences(valid_data, seq_len)
    X_test, y_test = create_sequences(test_data, seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, seq_len)
    
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-1,:])
        output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3, l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]

        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, n_predict, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)
    
    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
        
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, 0, -1], label='Actual')
        plt.plot(time_steps, output_org[:, 0, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-n_predict:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1}_{test_length})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, 0, j], label='Actual')
            ax.plot(time_steps, output_org[:, 0, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + n_predict + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-n_predict:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1}_{test_length})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2

def predict_future(model, X_test, n_predict, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
        return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    merge_future_predictions = None

    for recent_input_sequence in last_sequences:
        future_predictions = []

        for i in range(n_predict):
            # Generate a prediction
            recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
            with torch.no_grad():
                input_seq = torch.Tensor(recent_input_sequence).to(device)
                output = model(input_seq).cpu().numpy() 

                future_prediction = output[0, 0, :]

            # Append the prediction to the future_predictions list
            future_predictions.append(future_prediction)         
     
            # Update the input sequence with the new prediction, if not the last iteration
            if i < n_predict - 1:
                recent_input_sequence = update_sequence(recent_input_sequence, future_prediction, sequence_length)

            else:
                break
        
        future_predictions_array = np.array(future_predictions)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if merge_future_predictions is None:
            merge_future_predictions = future_predictions_inverse
            merge_future_predictions_org = future_predictions_array
        else:
            merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
            merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
    return merge_future_predictions, merge_future_predictions_org          
# def predict_future(model, X_test, y_test_unshifted, n_predict=5, forward=-1, n_last_sequence=1, scaler=None):
#     n_features = X_test.shape[2]
#     sequence_length = X_test.shape[1]
#     torch.cuda.empty_cache()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
#     model.eval()

#     def update_sequence(recent_input_sequence, future_next_prediction, sequence_length):
#         return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_next_prediction[np.newaxis, np.newaxis, :]], axis=1)
        
#     # def new_sequence(last_sequences, y_test, sequence_length):
#     #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
#     # Prepare the most recent input sequence
#     # x_test_sequences = X_test[-(n_last_sequence):, :, :]
#     # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
#     # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
#     y_test_unshifted_sequences = y_test_unshifted[-(n_last_sequence-forward):, :, :]
    
#     last_sequences = X_test[-(n_last_sequence):, :, :]
#     last_sequences = torch.Tensor(last_sequences)
    
#     merge_future_predictions = None

#     for idx, recent_input_sequence in enumerate(last_sequences):

#         future_predictions = []

#         for i in range(n_predict):
#             # Generate a prediction
#             recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
#             with torch.no_grad():
#                 input_seq = torch.Tensor(recent_input_sequence).to(device)
#                 output = model(input_seq).cpu().numpy() 
#                 # Use only the last feature from output and substitute other features with those from recent_input_sequence
#                 last_feature_prediction = output[0, 0, -1:]  # Shape should be (1,)
                
#                 future_prediction = output[0, 0, :]

#             # Append the prediction to the future_predictions list
#             future_predictions.append(future_prediction)         
     
#             # Update the input sequence with the new prediction, if not the last iteration
#             if idx + i + 1 < len(y_test_unshifted_sequences):
                
#                 other_features_from_y_test_unshifted = y_test_unshifted_sequences[idx+i+1, -1:, :-1]  # Shape should be (n_features-1,)
#                 future_next_prediction = np.concatenate([other_features_from_y_test_unshifted.flatten(), last_feature_prediction])
                
#                 recent_input_sequence = update_sequence(recent_input_sequence, future_next_prediction, sequence_length)
#             else:
#                 break
        
#         future_predictions_array = np.array(future_predictions)
#         future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

#         if merge_future_predictions is None:
#             merge_future_predictions = future_predictions_inverse
#             merge_future_predictions_org = future_predictions_array
#         else:
#             merge_future_predictions = np.vstack((np.round(merge_future_predictions, 5), np.round(future_predictions_inverse, 5)))
#             merge_future_predictions_org = np.vstack((np.round(merge_future_predictions_org, 5), np.round(future_predictions_array, 5)))

        
#     return merge_future_predictions, merge_future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, start_date_short=None, end_date_short=None,
                  valid_size=0.5, n_predict=5, seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # Generate random hyperparameters and parameters
        seq_len = random.choice(range(5, 16))
        nlayers = random.choice(range(1, 5))
        nneurons = random.choice(range(32, 501))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0.1])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(300, 500))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001, 0.0005, 0.001])
        patience = random.choice(range(20, 21))
        min_delta = random.choice([0.0001])
        l2_regularization = random.choice([0.0001, 0.001])
        
        # # Generate random hyperparameters and parameters
        # seq_len = random.choice(range(10, 11))
        # nlayers = random.choice(range(2, 3))
        # nneurons = random.choice(range(100, 101))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # dropout = random.choice([0])
        # optimizer = random.choice([torch.optim.Adam, torch.optim.AdamW, torch.optim.Adagrad])
        # n_epochs = random.choice(range(300, 500))
        # batch_size = random.choice(range(256, 512))
        # learning_rate = random.choice([0.0001])
        # patience = random.choice(range(20, 21))
        # min_delta = random.choice([0.0001])
        # l2_regularization = random.choice([0])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, seq_len=seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, seq_len= prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short, 
                                                                                        end_date_short=end_date_short, seq_len=seq_len, target_col=target_col, forward=forward)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data, test_data_short=test_data_short, seq_len=seq_len)
        
        input_shape = (X_train.shape[0], seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"seq_len": seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_predict": n_predict, "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # # Inverse transform the y_test to the original scale
        # test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        # test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        
        # test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        # test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)
        
        # Inverse transform the y_test to the original scale 
        test_scaler = StandardScaler().fit(test_data_unnormalized)
        
        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized)
        
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_predict > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test, n_predict=n_predict,
                                                        n_last_sequence=n_last_sequence, scaler=test_scaler)
        # print(f"Future Predictions (Trial {trial+1}): {future_predictions.shape}")
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(n_predict*n_predict):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")
        
        # Generate future predictions
        if n_last_sequence > 0:
            short_future_predictions, short_future_predictions_org = predict_future(loaded_model, X_test_short, n_predict=n_predict,
                                                                                    n_last_sequence=n_last_sequence, scaler=test_scaler_short)
        short_future_predictions = np.squeeze(short_future_predictions)
        short_future_predictions_org = np.squeeze(short_future_predictions_org) 

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label, test_length="long",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)
            
            plot_predictions(loaded_model, X_test_short, y_test_short, trial, n_predict, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler_short, col_label=col_label, test_length="short",
                             future_predictions=short_future_predictions if len(short_future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > n_predict:
                    y_true_all_features = y_test_org[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence+1-i):-((n_last_sequence+1-i)-n_predict), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                # else:
                #     y_true_all_features = y_test_org[-(n_last_sequence-i):, -1]
                #     y_true = y_true_all_features[:, -1]
                #     y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_pred = y_pred_all_features[:, -1]

                #     # Inverse transform the y_test to the original scale
                #     y_actual_all_features = y_test[-(n_last_sequence-i):, -1]
                #     y_actual = y_actual_all_features[:, -1]
                #     y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# modified_2_short test data_+data_shift
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import yfinance as yf
import timeit
import random
from torch.nn.modules.transformer import TransformerEncoderLayer, TransformerEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def create_sequences(data, X_seq_len, y_seq_len):
    X = []
    y = []

    for i in range(len(data) - X_seq_len - y_seq_len + 1):
        X.append(data[i : i + X_seq_len])
        y.append(data[i + X_seq_len : i + X_seq_len + y_seq_len])

    return np.array(X), np.array(y)

def create_differences(data):
    # Check if the required columns exist in the data
    if set(['Open', 'Close', 'High', 'Low']).issubset(data.columns):
        # Create new columns
        data['Diff_1'] = data['Close'] - data['Open']
        data['Diff_2'] = data['High'] - data['Low']

        # Drop unnecessary columns
        data = data.drop(columns=['Open', 'Low', 'High'])
        
    else:
        print("One or more of the required columns ('Open', 'Close', 'High', 'Low') are not present in the input dataframe.")
    
    return data

def calculate_indicators(data, rsi_period=14, short_ema_period=12, long_ema_period=26, signal_period=9, vol_period=20):
    # Check if 'Close' column exists in the data
    if 'Close' in data.columns:
        # Calculate daily returns
        data['Diff_1'] = data['Close'] - data['Open']
        data['Diff_2'] = data['High'] - data['Low']

        # Drop unnecessary columns
        data = data.drop(columns=['Open', 'Low', 'High'])
        
        data['Return'] = data['Close'].pct_change()

        # Calculate RSI
        delta = data['Close'].diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0

        average_gain = up.rolling(window=rsi_period).mean()
        average_loss = abs(down.rolling(window=rsi_period).mean())

        rs = average_gain / average_loss

        data['RSI'] = 100 - (100 / (1 + rs))

        # Calculate MACD Line: (12-day EMA - 26-day EMA)
        EMA_short = data['Close'].ewm(span=short_ema_period).mean() 
        EMA_long = data['Close'].ewm(span=long_ema_period).mean() 
        data['MACD_Line'] = EMA_short - EMA_long

        # Calculate Signal Line: a n-day MA of MACD Line 
        data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

        # Calculate Volatility as rolling standard deviation of log returns
        data["Log_Return"] = np.log(data["Close"]).diff()
        data["Volatility"] = data["Log_Return"].rolling(window=vol_period).std()

    else:
         print("'Close' column is not present in the input dataframe.")
    
    return data


def prepare_data_whole(data, seq_len, target_col, scaler=StandardScaler, valid_size=0.2, forward=-1):
    if isinstance(target_col, int):
        target_col_name = data.columns[target_col]
    else:
        target_col_name = target_col
        
    data = data.copy()
    data['Target'] = data[target_col_name].shift(forward)
    data.dropna(inplace=True)
    data = data.drop(target_col_name, axis=1)
    
    data[data.columns] = scaler().fit_transform(data)
    
    train_data, test_valid_data = train_test_split(data, test_size=valid_size, shuffle=False)
    valid_data, test_data = train_test_split(test_valid_data, test_size=0.5, shuffle=False)

    return prepare_data_common(train_data, valid_data, test_data, seq_len)

def fetch_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    # return data.drop(['Adj Close', 'Volume'], axis=1)
    return data.drop('Adj Close', axis=1)

def prepare_data_separate(train_data_list, valid_data_list, X_seq_len, y_seq_len, target_col, symbol, start_date, end_date, 
                          start_date_short=None, end_date_short=None, scaler=StandardScaler(), forward=-1):
    if isinstance(target_col, int):
        target_col_name = train_data_list[0].columns[target_col]
    else:
        target_col_name = target_col

    # Scale train data
    combined_train_data = None
    for train_data in train_data_list:
        # train_data = train_data.copy()

        # # Create separate dataframes for prices and volume
        # train_data_reshaped = train_data.values.reshape(-1, 1)

        # train_data_transformed = scaler.fit_transform(train_data_reshaped)

        # # Reshape it back to original shape.
        # train_data[train_data.columns] = train_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # train_data['Target'] = train_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # train_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # train_data.drop(target_col_name, axis=1, inplace=True)
        
        
        train_data = train_data.copy()
        train_data = calculate_indicators(train_data)
        
        train_data['Target'] = train_data[target_col_name].shift(forward)
        train_data.dropna(inplace=True)
        train_data = train_data.drop(target_col_name, axis=1)
        
        train_data[train_data.columns] = scaler.fit_transform(train_data)

        
        if combined_train_data is None:
            combined_train_data = train_data
        else:
            combined_train_data = pd.concat([combined_train_data, train_data], ignore_index=True)
    # Scale valid data
    combined_valid_data = None
    for valid_data in valid_data_list:
        
        # valid_data = valid_data.copy()

        # # Create separate dataframes for prices and volume
        # valid_data_reshaped = valid_data.values.reshape(-1, 1)

        # valid_data_transformed = scaler.fit_transform(valid_data_reshaped)

        # # Reshape it back to original shape.
        # valid_data[valid_data.columns] = valid_data_transformed.reshape(-1, 4)

        # # Shift target column by forward steps.
        # valid_data['Target'] = valid_data[target_col_name].shift(forward)

        # # Drop NA values if there are any due to shifting.
        # valid_data.dropna(inplace=True)

        # # Drop original target column after creating shifted Target.
        # valid_data.drop(target_col_name, axis=1, inplace=True)
        
        valid_data = valid_data.copy()
        valid_data = calculate_indicators(valid_data)
        
        valid_data['Target'] = valid_data[target_col_name].shift(forward)
        valid_data.dropna(inplace=True)
        valid_data = valid_data.drop(target_col_name, axis=1)
        valid_data[valid_data.columns] = scaler.fit_transform(valid_data)
        
        if combined_valid_data is None:
            combined_valid_data = valid_data
        else:
            combined_valid_data = pd.concat([combined_valid_data, valid_data], ignore_index=True)
            
    # Fetch a fresh copy of the test data
    test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)
    
    # test_data_unshifted = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_unshifted_reshaped = test_data_unshifted.values.reshape(-1, 1)

    # test_data_unshifted_transformed = scaler.fit_transform(test_data_unshifted_reshaped)

    # # Reshape it back to original shape.
    # test_data_unshifted[test_data_unshifted.columns] = test_data_unshifted_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]

    # # Drop NA values if there are any due to shifting.
    # test_data_unshifted.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_unshifted.drop(target_col_name, axis=1, inplace=True)
 
    # test_data = test_data.copy()

    # # Create separate dataframes for prices and volume
    # test_data_reshaped = test_data.values.reshape(-1, 1)

    # test_data_transformed = scaler.fit_transform(test_data_reshaped)

    # # Reshape it back to original shape.
    # test_data[test_data.columns] = test_data_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data['Target'] = test_data[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data.drop(target_col_name, axis=1, inplace=True)
    
    # # Fetch a fresh copy of the test data
    # test_data = fetch_data(symbol=symbol,start_date=start_date,end_date=end_date)
    # Scale test data
    test_data_unnormalized = test_data.copy()
    test_data_unnormalized = calculate_indicators(test_data_unnormalized)
    
    test_data_unnormalized['Target'] = test_data_unnormalized[target_col_name]
    test_data_unnormalized.dropna(inplace=True)
    test_data_unnormalized = test_data_unnormalized.drop(target_col_name, axis=1)

    # test_data_unshifted = test_data.copy()
    # test_data_unshifted['Target'] = test_data_unshifted[target_col_name]
    # test_data_unshifted.dropna(inplace=True)
    # test_data_unshifted = test_data_unshifted.drop(target_col_name, axis=1)
    # test_data_unshifted[test_data_unshifted.columns] = scaler.fit_transform(test_data_unshifted)

    # Scale test data
    test_data = test_data.copy()
    test_data = calculate_indicators(test_data)
    
    test_data[test_data.columns] = scaler.fit_transform(test_data)
    test_data['Target'] = test_data[target_col_name].shift(forward)
    test_data.dropna(inplace=True)
    test_data = test_data.drop(target_col_name, axis=1)    
    
    # Fetch a fresh copy of a short test data
    test_data_short = fetch_data(symbol=symbol,start_date=start_date_short,end_date=end_date_short)
    
    # Scale test data
    test_data_short_unnormalized = test_data_short.copy()
    test_data_short_unnormalized = calculate_indicators(test_data_short_unnormalized)
    
    test_data_short_unnormalized['Target'] = test_data_short_unnormalized[target_col_name]
    test_data_short_unnormalized.dropna(inplace=True)
    test_data_short_unnormalized = test_data_short_unnormalized.drop(target_col_name, axis=1)    

    # test_data_short = test_data_short.copy()

    # # Create separate dataframes for prices and volume
    # test_data_short_reshaped = test_data_short.values.reshape(-1, 1)

    # test_data_short_transformed = scaler.fit_transform(test_data_short_reshaped)

    # # Reshape it back to original shape.
    # test_data_short[test_data_short.columns] = test_data_short_transformed.reshape(-1, 4)

    # # Shift target column by forward steps.
    # test_data_short['Target'] = test_data_short[target_col_name].shift(forward)

    # # Drop NA values if there are any due to shifting.
    # test_data_short.dropna(inplace=True)

    # # Drop original target column after creating shifted Target.
    # test_data_short.drop(target_col_name, axis=1, inplace=True)
    
    # Scale test data
    test_data_short = test_data_short.copy()
    test_data_short = calculate_indicators(test_data_short)
    test_data_short[test_data_short.columns] = scaler.fit_transform(test_data_short)
    test_data_short['Target'] = test_data_short[target_col_name].shift(forward)
    test_data_short.dropna(inplace=True)
    test_data_short = test_data_short.drop(target_col_name, axis=1) 
   
    return combined_train_data, combined_valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, X_seq_len, y_seq_len
    
def prepare_data_common(train_data, valid_data, test_data, test_data_short, X_seq_len, y_seq_len):
    # Create sequences
    X_train, y_train = create_sequences(train_data, X_seq_len, y_seq_len)
    X_valid, y_valid = create_sequences(valid_data, X_seq_len, y_seq_len)
    X_test, y_test = create_sequences(test_data, X_seq_len, y_seq_len)
    X_test_short, y_test_short = create_sequences(test_data_short, X_seq_len, y_seq_len)
    
    
    # Convert to PyTorch tensors
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_valid = torch.Tensor(X_valid)
    y_valid = torch.Tensor(y_valid)
    X_test = torch.Tensor(X_test)
    y_test = torch.Tensor(y_test)
    X_test_short = torch.Tensor(X_test_short)
    y_test_short = torch.Tensor(y_test_short)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short

class LSTMRegression(nn.Module):
    def __init__(self, input_shape, nlayers=2,
                 nneurons=64, dropout=0.2, y_seq_len=5):
        super(LSTMRegression, self).__init__()

        self.dropout = nn.Dropout(dropout)
        self.hidden_layers = nn.ModuleList()
        self.y_seq_len = y_seq_len
        
        for _ in range(nlayers):
            lstm_layer = nn.LSTM(input_size=input_shape[-1] if _ == 0 else nneurons,
                                 hidden_size=nneurons,
                                 batch_first=True)
            self.hidden_layers.append(lstm_layer)
            self.hidden_layers.append(self.dropout)

        # Output layer
        self.output = nn.Linear(nneurons, input_shape[-1])

    def forward(self, x):
        for i in range(0,len(self.hidden_layers),2):  # Step size of 2 because we have an LSTM and Dropout at each step.
          x,_=self.hidden_layers[i](x)
          x=self.hidden_layers[i+1](x)   # Applying dropout after each LSTM layer

        output=self.output(x[:,-self.y_seq_len:,:])
        # output = output.unsqueeze(1)
        
        return output
    
def train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=torch.optim.Adam,
                batch_size=32, patience=10, min_delta=0.0001, learning_rate=1e-3,
                l2_regularization=0.0001, max_norm=1.0, nan_patience=1):

    # Enable cuDNN
    torch.backends.cudnn.enabled = True
    torch.cuda.empty_cache()
    optimizer = optimizer(model.parameters(), lr=learning_rate, weight_decay=l2_regularization)
    criterion = nn.MSELoss()

    # Setup GPU device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Put model on GPU
    model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    X_valid = X_valid.to(device)
    y_valid = y_valid.to(device)
    valid_dataset = TensorDataset(X_valid, y_valid)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    # print(next(model.parameters()).device)
    # print(X_train.device)

    # Early stopping parameters
    patience = patience  # number of epochs with no improvement
    best_val_loss = float('inf')

    train_losses = []
    val_losses = []
    early_stopping_counter = 0

    # NaN stopping parameters
    nan_counter = 0
    stopped_early = False

    for epoch in range(n_epochs):
        # print(next(model.parameters()).device)
        # print(X_train.device)
        model.train()
        epoch_train_losses = []
        for batch_X_train, batch_y_train in train_loader:
            batch_X_train = batch_X_train.to(device)
            batch_y_train = batch_y_train.to(device)

            optimizer.zero_grad()
            output = model(batch_X_train)
            loss = criterion(output, batch_y_train)

            if torch.isnan(loss):
                nan_counter += 1
            else:
                nan_counter = 0

            if nan_counter >= nan_patience:
                print(f"Training stopped early at epoch {epoch} due to NaNs in loss")
                stopped_early = True
                break

            loss.backward()
            # Add the gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
            optimizer.step()

            epoch_train_losses.append(loss.item())

        # Break the outer loop if NaN stopping was triggered
        if nan_counter >= nan_patience:
            break

        train_losses.append(np.mean(epoch_train_losses))

        model.eval()
        epoch_val_losses = []
        with torch.no_grad():
            for batch_X_valid, batch_y_valid in valid_loader:
                batch_X_valid = batch_X_valid.to(device)
                batch_y_valid = batch_y_valid.to(device)

                valid_output = model(batch_X_valid)
                val_loss = criterion(valid_output, batch_y_valid)
                epoch_val_losses.append(val_loss.item())

        val_losses.append(np.mean(epoch_val_losses))

        # Print the running output
        print(f"Epoch {epoch}: Train Loss = {train_losses[-1]:.4f}, Val Loss = {val_losses[-1]:.4f}")

        # Early stopping
        if val_losses[-1] < best_val_loss - min_delta:
            best_val_loss = val_losses[-1]
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1

        if early_stopping_counter >= patience:
            print("Early stopping triggered due to no improvement in validation loss.")
            break

    return train_losses, val_losses, stopped_early

def evaluate_model(model, X, y, use_target_col=True):
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    X = X.to(device)
    y = y.to(device)

    with torch.no_grad():
        y_pred = model(X)

        # Reshape the tensors to 2D and move them back to the CPU before computing metrics
        y = y.view(-1, y.shape[-1]).cpu()
        y_pred = y_pred.view(-1, y_pred.shape[-1]).cpu()

        if use_target_col:
            y = y[:,-1] # Pick the last column (target column)
            y_pred = y_pred[:,-1]

        mse = mean_squared_error(y, y_pred)
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)

    return mse, mae, r2

def plot_results(train_losses, val_losses, trial, save_directory=None):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss', 'Valid Loss'])
    plt.title(f'Train and Valid Losses (Trial {trial+1})')

    if save_directory:
        save_path = os.path.join(save_directory, f"loss_plot_trial_{trial}.png")
        plt.savefig(save_path)

    plt.show()
    
def inverse_transform_wrapper(data, orgshape, scaler):
    data_reshaped = data.reshape(-1, data.shape[-1])
    data_inv = scaler.inverse_transform(data_reshaped)
    data_inv_origshape = data_inv.reshape(orgshape)
    return data_inv_origshape

def plot_predictions(model, X_test, y_test, trial, y_seq_len=None, use_target_col=True, save_directory=None, 
                     future_predictions=None, scaler=None, col_label=None, test_length=None):
    torch.cuda.empty_cache()
    # Get n_features from X_test
    n_features = X_test.shape[2]
    
    # Move the model and input tensor to the same device.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    X_test = X_test.to(device)

    # Run the model on the input tensor and move the predictions back to the CPU, if needed.
    model.eval()
    with torch.no_grad():
        output = model(X_test).cpu()
   
    y_test_org = inverse_transform_wrapper (y_test, y_test.shape, scaler=scaler)
    output_org = inverse_transform_wrapper (output, output.shape, scaler=scaler)
    
    # If given, transform future predictions back to the original scale
    if future_predictions is not None:
        gap = 0

    else:
        print("No future predictions found.")
        gap = 0
    
    # If future_predictions is not None, plot the future predictions
    # If use_target_col is True, only plot the target column, otherwise plot all feature columns
    if use_target_col:
        # the existing time steps first
        time_steps = list(range(len(y_test_org)))
        
        plt.figure(figsize=(15, 8))
        plt.plot(time_steps, y_test_org[:, :, -1], label='Actual')
        plt.plot(time_steps, output_org[:, :, -1], label='Predicted')
        
        # generate the future time steps
        future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + y_seq_len + gap))
        print('Plotting future predictions...')
        print("future_time_steps:", future_time_steps)
        last_future_prediction = future_predictions[-y_seq_len:]
        print("future_predictions:", last_future_prediction[:, -1])
        plt.plot(future_time_steps, last_future_prediction[:, -1], label='Future Predicted')

        plt.xlabel('Time Step')
        plt.ylabel('Value')
        # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
        plt.title(f'Actual and Predicted Values for {col_label[-1]} (Trial {trial+1}_{test_length})')
        plt.legend()
        if save_directory:
            save_path = os.path.join(save_directory, f"predictions_plot_target_trial_{trial+1}_{test_length}.png")
            plt.savefig(save_path)
        plt.show()
    else:
        for j in range(n_features):
            time_steps = list(range(len(y_test_org)))
            fig, ax = plt.subplots(figsize=(15, 8))
            ax.plot(time_steps, y_test_org[:, :, j], label='Actual')
            ax.plot(time_steps, output_org[:, :, j], label='Predicted')            
          
            # Generate the future time steps
            future_time_steps = list(range(len(y_test_org) + gap, len(y_test_org) + y_seq_len + gap))
            print('Plotting future predictions...')
            print("future_time_steps:", future_time_steps)
            last_future_prediction = future_predictions[-y_seq_len:]
            print("future_predictions:", last_future_prediction[:, j])
            plt.plot(future_time_steps, last_future_prediction[:, j], label=f'Future Predicted for {col_label[j]}')

            ax.set_xlabel('Time Step')
            ax.set_ylabel('Value')
            # ax.set_title(f'Actual and Predicted Values for Variable {j + 1} (Trial {trial+1})')
            plt.title(f'Actual and Predicted Values for {col_label[j]} (Trial {trial+1}_{test_length})')
            ax.legend()
            if save_directory:
                save_path = os.path.join(save_directory, f"predictions_plot_var_{j + 1}_trial_{trial}-{test_length}.png")
                plt.savefig(save_path)
            plt.show()

def calculate_metrics(y_true: np.ndarray , y_pred: np.ndarray):
    mse = mean_squared_error(y_true=y_true,y_pred=y_pred)
    mae = mean_absolute_error(y_true=y_true,y_pred=y_pred)
    r2 = r2_score(y_true=y_true,y_pred=y_pred)
    
    return mse, mae, r2
          
def predict_future(model, X_test, n_last_sequence=1, scaler=None):
    n_features = X_test.shape[2]
    sequence_length = X_test.shape[1]
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    # def update_sequence(recent_input_sequence, future_update, sequence_length):
    #     return np.concatenate([recent_input_sequence[:, -(sequence_length-1):, :], future_update[np.newaxis, np.newaxis, :]], axis=1)
        
    # def new_sequence(last_sequences, y_test, sequence_length):
    #     return np.concatenate([last_sequences[:, -(sequence_length-1):, :], y_test[:, :, :]], axis=1)
        
    # Prepare the most recent input sequence
    # x_test_sequences = X_test[-(n_last_sequence):, :, :]
    # y_test_sequences = y_test[-(n_last_sequence):, :, :]
    
    # merge_sequences = new_sequence(x_test_sequences, y_test_sequences, sequence_length)
    last_sequences = X_test[-(n_last_sequence):, :, :]
    last_sequences = torch.Tensor(last_sequences)
    
    future_predictions = None

    for recent_input_sequence in last_sequences:
        # Generate a prediction
        recent_input_sequence = recent_input_sequence.reshape(1, sequence_length, n_features) 
        with torch.no_grad():
            input_seq = torch.Tensor(recent_input_sequence).to(device)
            output = model(input_seq).cpu().numpy() 

            future_prediction = output[0, :, :]
            # future_update = output[0, 0, 1]
    
        # # Update the input sequence with the new prediction, if not the last iteration
        # recent_input_sequence = update_sequence(recent_input_sequence, future_update, sequence_length)
       
        future_predictions_array = np.array(future_prediction)
        future_predictions_inverse = inverse_transform_wrapper(future_predictions_array, future_predictions_array.shape, scaler=scaler)

        if future_predictions is None:
            future_predictions = future_predictions_inverse
            future_predictions_org = future_predictions_array
        else:
            future_predictions = np.vstack((np.round(future_predictions, 5), np.round(future_predictions_inverse, 5)))
            future_predictions_org = np.vstack((np.round(future_predictions_org, 5), np.round(future_predictions_array, 5)))
        
    return future_predictions, future_predictions_org

def random_search(data, target_col=None, n_trials=1, n_top_models=1,
                   model_save=True, save_directory=None, plot_loss=True, predict_plot=True, 
                  future_plot=True, overall_future_plot=True, future_predictions=None, 
                  use_target_col=True, train_data_list=None, valid_data_list=None,
                  symbol=None, start_date=None, end_date=None, start_date_short=None, end_date_short=None,
                  valid_size=0.5, X_seq_len=10, y_seq_len=5, n_last_sequence=1, forward=-1):
    
    if save_directory and not os.path.exists(save_directory):
        os.makedirs(save_directory)

    results_df = pd.DataFrame(columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
    
    top_models = []
    all_future_predictions = [] # Initialize the list to save all future predictions from each trial
    all_future_metrics =[]
    all_overall_future_metrics = []

    for trial in range(n_trials):
        print(f"Trial {trial + 1} of {n_trials}")
        start = timeit.default_timer()
    
        # Generate random hyperparameters and parameters
        X_seq_len = random.choice(range(10, 11))
        y_seq_len = random.choice(range(4, 5))
        nlayers = random.choice(range(1, 3))
        nneurons = random.choice(range(200, 301))
        # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        dropout = random.choice([0])
        optimizer = random.choice([torch.optim.Adam])
        n_epochs = random.choice(range(500, 1001))
        batch_size = random.choice(range(256, 512))
        learning_rate = random.choice([0.0001])
        patience = random.choice(range(5, 6))
        min_delta = random.choice([0.0001])
        l2_regularization = random.choice([0])
        
        # # Generate random hyperparameters and parameters
        # seq_len = random.choice(range(20, 61))
        # nlayers = random.choice(range(1, 6))
        # nneurons = random.choice(range(100, 256))
        # # activation_function = random.choice([torch.nn.Tanh(), torch.nn.Sigmoid(), torch.nn.ELU(), torch.nn.ReLU(), torch.nn.LeakyReLU(negative_slope=0.01)])
        # dropout = random.choice([0.1, 0.2, 0.3, 0.4])
        # optimizer = random.choice([torch.optim.Adam])
        # n_epochs = random.choice(range(300, 500))
        # batch_size = random.choice(range(128, 256))
        # learning_rate = random.choice([0.0001, 0.0005, 0.001])
        # patience = random.choice(range(5, 21))
        # min_delta = random.choice([0.0001, 0.0002])
        # l2_regularization = random.choice([0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1])

        # Prepare and preprocess the data
        if data is not None:
            X_train, y_train, X_valid, y_valid, X_test, y_test = prepare_data_whole(data=data, X_seq_len=X_seq_len, y_seq_len=y_seq_len,
                                                                target_col=target_col, valid_size=valid_size,forward=forward)

        if train_data_list is not None:
            train_data, valid_data, test_data, test_data_unnormalized, test_data_short, test_data_short_unnormalized, X_seq_len, y_seq_len = prepare_data_separate(train_data_list=train_data_list, valid_data_list=valid_data_list,
                                                                                        symbol=symbol,start_date=start_date,end_date=end_date, start_date_short=start_date_short, 
                                                                                        end_date_short=end_date_short, X_seq_len=X_seq_len, y_seq_len=y_seq_len, target_col=target_col)

            # Call prepare_data_common() with test_data_unnormalized
            X_train, y_train, X_valid, y_valid, X_test, y_test, X_test_short, y_test_short = prepare_data_common(train_data=train_data, valid_data=valid_data, test_data=test_data, 
                                                                                                                test_data_short=test_data_short, X_seq_len=X_seq_len, y_seq_len=y_seq_len)
        
        input_shape = (X_train.shape[0], X_seq_len, X_train.shape[2])
        
        # Initialize the model
        model = LSTMRegression(input_shape=input_shape, nlayers=nlayers, nneurons=nneurons, dropout=dropout, y_seq_len=y_seq_len)

        # Train the model
        train_losses, val_losses, stopped_early = train_model(model, X_train, y_train, X_valid, y_valid, n_epochs, optimizer=optimizer, 
                                                              batch_size=batch_size, patience=patience, min_delta=min_delta, learning_rate=learning_rate, l2_regularization=l2_regularization)
        # Check if training stopped early due to NaNs or not
        if stopped_early:
            print(f"Random search iteration {trial+1} stopped early due to NaNs in loss")
            # Using 'continue' here will skip the remaining statements of the current iteration and proceed to the next iteration
            continue

        if plot_loss:
            plot_results(train_losses, val_losses, trial, save_directory=save_directory)
                     
        # Evaluate the model on both train and test data
        train_mse, train_mae, train_r2 = evaluate_model(model, X_train, y_train)
        test_mse, test_mae, test_r2 = evaluate_model(model, X_test, y_test)
        
        # Add the results to the results dataframe
        params = {"X_seq_len": X_seq_len, "y_seq_len": y_seq_len, "nlayers": nlayers, "nneurons": nneurons, 
                  "dropout": dropout, "optimizer": optimizer, "n_epochs": n_epochs,
                  "batch_size": batch_size, "learning_rate": learning_rate,
                  "patience": patience, "min_delta": min_delta, "l2_regularization": l2_regularization,
                  "n_last_sequence": n_last_sequence, "forward": forward}        
       
        trial_results = [trial, params, round(train_mse, 5), round(train_mae, 5), round(train_r2, 5), round(test_mse, 5), round(test_mae, 5), round(test_r2, 5)]
        results_df.loc[len(results_df)] = trial_results

        if save_directory:
            results_df.to_csv(os.path.join(save_directory, f"results_{trial}.csv"))
            
        # initialize variables to store most recently saved model's path
        most_recent_save_path = None

        # Save the model
        if model_save:
            if save_directory:
                save_path = os.path.join(save_directory, f"model_trial_{trial}.pt")
            else:
                save_path = f"model_trial_{trial}.pt"
            torch.save(model, save_path)
            most_recent_save_path = save_path

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the most recently saved model
        if most_recent_save_path:
            loaded_model = torch.load(most_recent_save_path)
            loaded_model = loaded_model.to(device)
            loaded_model.eval()
        
        # # Inverse transform the y_test to the original scale
        # test_data_unnormalized_reshaped = test_data_unnormalized.values.reshape(-1, 1)  
        # test_scaler = StandardScaler().fit(test_data_unnormalized_reshaped)
        
        # test_data_short_unnormalized_reshaped = test_data_short_unnormalized.values.reshape(-1, 1)
        # test_scaler_short = StandardScaler().fit(test_data_short_unnormalized_reshaped)
        
        # Inverse transform the y_test to the original scale 
        test_scaler = StandardScaler().fit(test_data_unnormalized)
        
        test_scaler_short = StandardScaler().fit(test_data_short_unnormalized)
        
        # Get the column names
        col_label = test_data_unnormalized.columns

        # Generate future predictions
        if n_last_sequence > 0:
            future_predictions, future_predictions_org = predict_future(loaded_model, X_test,  
                                                        n_last_sequence=n_last_sequence, scaler=test_scaler)
        future_predictions_df = pd.DataFrame(future_predictions, columns=[f"Future_Predicted_{col_label[i]}" for i in range(X_test.shape[2])])
        future_predictions_all_features = future_predictions_df.iloc[-(y_seq_len*y_seq_len):]
        future_predictions_target = future_predictions_all_features.iloc[:, -1]

        # Create a DataFrame for future_predictions_target with a 'Trial' column
        future_predictions_target_df = future_predictions_target.to_frame(name='Future_Predicted_Target')
        future_predictions_target_df['Trial'] = trial + 1

        # Append the new DataFrame to the list
        all_future_predictions.append(future_predictions_target_df)

        # Concatenate all the future predictions into a single DataFrame
        all_future_predictions_df = pd.concat(all_future_predictions, axis=0)
        print(f"Future Predictions (Trial {trial+1}): {future_predictions_target_df}")

        # Plot prediction results
        if predict_plot:
            plot_predictions(loaded_model, X_test, y_test, trial, y_seq_len=y_seq_len, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler, col_label=col_label, test_length="long",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)
            
            plot_predictions(loaded_model, X_test=X_test_short, y_test=y_test_short, trial=trial, y_seq_len=y_seq_len, use_target_col=use_target_col,
                             save_directory=save_directory, scaler=test_scaler_short, col_label=col_label, test_length="short",
                             future_predictions=future_predictions if len(future_predictions) > 0 else None)

        # Inverse transform the y_test to the original scale
        y_test_org = inverse_transform_wrapper(y_test, y_test.shape, scaler=test_scaler)

        future_metrics_trial = []

        # Initialize accumulators
        accumulated_y_true_all_features = []
        accumulated_y_true = []
        accumulated_y_pred_all_features = []
        accumulated_y_pred = []
        accumulated_y_actual_all_features = []
        accumulated_y_actual = []
        accumulated_y_predicted_all_features = []
        accumulated_y_predicted = []


        for i in range(n_last_sequence):
            # Calculate metrics for the last sequence of true labels vs predicted labels
            if y_test_org.shape[0] >= n_last_sequence:
                if n_last_sequence-i > y_seq_len:
                    y_true_all_features = y_test_org[-(n_last_sequence-i):-((n_last_sequence-i)-y_seq_len), -1]
                    y_true = y_true_all_features[:, -1]
                    y_pred_all_features = np.array(future_predictions[-(y_seq_len*(n_last_sequence-i)):-(y_seq_len*(n_last_sequence-(i+1)))])
                    y_pred = y_pred_all_features[:, -1]

                    # Inverse transform the y_test to the original scale
                    y_actual_all_features = y_test[-(n_last_sequence-i):-((n_last_sequence-i)-y_seq_len), -1]
                    y_actual = y_actual_all_features[:, -1]
                    y_predicted_all_features = np.array(future_predictions_org[-(y_seq_len*(n_last_sequence-i)):-(y_seq_len*(n_last_sequence-(i+1)))])
                    y_predicted = y_predicted_all_features[:, -1]
                # else:
                #     y_true_all_features = y_test_org[-(n_last_sequence+1-i):, -1]
                #     y_true = y_true_all_features[:, -1]
                #     y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_pred = y_pred_all_features[:, -1]

                #     # Inverse transform the y_test to the original scale
                #     y_actual_all_features = y_test[-(n_last_sequence+1-i):, -1]
                #     y_actual = y_actual_all_features[:, -1]
                #     y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
                #     y_predicted = y_predicted_all_features[:, -1]

        # for i in range(n_last_sequence-n_predict):
        #     # Calculate metrics for the last sequence of true labels vs predicted labels
        #     if y_test_unshifted.shape[0] >= n_last_sequence:
        #         if n_last_sequence-i > n_predict:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):-((n_last_sequence-i)-n_predict), -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-(n_predict*(n_last_sequence-(i+1)))])
        #             y_predicted = y_predicted_all_features[:, -1]
        #         else:
        #             y_true_all_features = y_test_unshifted_org[-(n_last_sequence-i):, -1]
        #             y_true = y_true_all_features[:, -1]
        #             y_pred_all_features = np.array(future_predictions[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_pred = y_pred_all_features[:, -1]

        #             # Inverse transform the y_test to the original scale
        #             y_actual_all_features = y_test_unshifted[-(n_last_sequence-i):, -1]
        #             y_actual = y_actual_all_features[:, -1]
        #             y_predicted_all_features = np.array(future_predictions_org[-(n_predict*(n_last_sequence-i)):-((n_predict*(n_last_sequence-(i+1)))-((n_last_sequence-i)-(n_predict)))])
        #             y_predicted = y_predicted_all_features[:, -1]

                # Add these lines inside both conditions above, after calculating y_* variables.
                accumulated_y_true_all_features.append(y_true_all_features)
                accumulated_y_true.append(y_true)
                accumulated_y_pred_all_features.append(y_pred_all_features)
                accumulated_y_pred.append(y_pred)
                accumulated_y_actual_all_features.append(y_actual_all_features)
                accumulated_y_actual.append(y_actual)
                accumulated_y_predicted_all_features.append(y_predicted_all_features)
                accumulated_y_predicted.append(y_predicted)

                # Calculate metrics for the last sequence of true labels vs predicted labels
                mse_org, mae_org, r2_org = calculate_metrics(y_pred, y_true)
                mse_org_all_features, mae_org_all_features, r2_org_all_features = calculate_metrics(y_pred_all_features, y_true_all_features)
                mse, mae, r2 = calculate_metrics(y_predicted, y_actual)
                mse_all_features, mae_all_features, r2_all_features = calculate_metrics(y_predicted_all_features, y_actual_all_features)
                # print(f"y_pred: {y_pred}, y_true: {y_true}")

                residual = y_true - y_pred
                error_percentage = (residual/y_true)*100
                average_error_percentage = np.mean(error_percentage)
        
                # Convert arrays to lists for better CSV saving
                y_true_list = y_true.tolist()   
                y_pred_list = y_pred.tolist()
                residual_list = residual.tolist()
                error_percentage_list = error_percentage.tolist()
                
                # Round values for better readability if desired
                y_true_list_rounded = [round(value ,4) for value in y_true_list]
                y_pred_list_rounded = [round(value ,4) for value in y_pred_list]
                residual_list_rounded=[round(value ,4) for value in residual_list]
                error_percentage_list_rounded=[round(value ,2) for value in error_percentage_list]
                
                # Save future MSE and R2, actual values, predicted values, and residuals
                future_metrics = {
                    "Trial": [trial],
                    "Future MSE (org)": [round(mse_org, 5)],
                    "Future MAE (org)": [round(mae_org, 5)],
                    "Future R2 (org)": [round(r2_org, 5)],
                    "Future MSE (org all features)": [round(mse_org_all_features, 5)],
                    "Future MAE (org all features)": [round(mae_org_all_features, 5)],
                    "Future R2 (org all features)": [round(r2_org_all_features, 5)],
                    "Future MSE": [round(mse, 5)],
                    "Future MAE": [round(mae, 5)],
                    "Future R2": [round(r2, 5)],
                    "Future MSE (all features)": [round(mse_all_features, 5)],
                    "Future MAE (all features)": [round(mae_all_features, 5)],
                    "Future R2 (all features)": [round(r2_all_features, 5)],
                    "Actual": [y_true_list_rounded],
                    "Predicted": [y_pred_list_rounded],
                    "Residual": [residual_list_rounded],
                    "Error Percentage": [error_percentage_list_rounded],
                    "Average Error Percentage": [round(average_error_percentage, 2)]
                }

                future_metrics_df = pd.DataFrame(future_metrics)

                # Add an index column that represents each iteration
                future_metrics_df['Trial'] = trial + 1
                future_metrics_df['Index'] = i + 1

            future_metrics_trial.append(future_metrics_df)

            # Plot the actual and predicted values for the last sequence of true labels vs predicted labels
            # Concatenate y_pred and future_predictions_target along rows
            if future_plot:
                
                print(f"Future MSE: {mse:.5f}, Future MAE: {mae:.5f}, Future R2: {r2:.5f}, Future MSE (all features): {mse_all_features:.5f}, "
                f"Future MAE (all features): {mae_all_features:.5f}, Future R2 (all features): {r2_all_features:.5f}, Future MSE (org): {mse_org:.5f}, "
                f"Future R2: {r2:.5f}, Average Error Percentage: {average_error_percentage:.3f}")

                combined_predictions = np.concatenate((y_pred, future_predictions_target))

                # Create a new figure
                plt.figure(figsize=(15, 8))
                plt.plot(y_true, label='Actual')

                # Plot combined predictions (past + future)
                plt.plot(combined_predictions, label='Predicted')

                # Add labels and title
                plt.xlabel('Time Step')
                plt.ylabel('Value')
                # plt.title(f'Actual and Predicted Values for Target Variable (Trial {trial+1})')
                plt.title(f'Actual and Future_Predicted Values for {col_label[-1]} (Trial {trial+1}, Index {i+1})')
                plt.legend()
                if save_directory:
                    save_path = os.path.join(save_directory, f"future_predictions_plot_target_trial_{trial+1}_prdict_{i+1}.png")
                    plt.savefig(save_path)
                plt.show()

        # Concatenate all the results into a single DataFrame after each trial
        all_future_metrics_trial_df = pd.concat(future_metrics_trial)

        # Reset index of final DataFrame for clarity after each trial and save it separately
        all_future_metrics_trial_df.reset_index(drop=True,inplace=True)

        all_future_metrics.append(all_future_metrics_trial_df)

        # Concatenate dataframes from all trials into a final dataframe.
        all_future_metric_finals=pd.concat(all_future_metrics,axis=0)

        # After your loop, convert accumulators into numpy arrays
        accumulated_y_true_all_features = np.concatenate(accumulated_y_true_all_features)
        accumulated_y_true = np.concatenate(accumulated_y_true)
        accumulated_y_pred_all_features = np.concatenate(accumulated_y_pred_all_features)
        accumulated_y_pred =np.concatenate (accumulated_y_pred )
        accumulated_y_actual_all_features = np.concatenate(accumulated_y_actual_all_features)
        accumulated_y_actual = np.concatenate(accumulated_y_actual)
        accumulated_y_predicted_all_features = np.concatenate(accumulated_y_predicted_all_features)
        accumulated_y_predicted = np.concatenate(accumulated_y_predicted)

        # Calculate overall metrics
        overall_mse_org, overall_mae_org, overall_r2_org= calculate_metrics(accumulated_y_pred ,accumulated_y_true)
        overall_mse_org_all_features, overall_mae_org_all_features, overall_r2_org_all_features = calculate_metrics(accumulated_y_pred_all_features ,accumulated_y_true_all_features)
        overall_mse, overall_mae, overall_r2 = calculate_metrics(accumulated_y_predicted ,accumulated_y_actual)
        overall_mse_all_features, overall_mae_all_features, overall_r2_all_features = calculate_metrics(accumulated_y_predicted_all_features ,accumulated_y_actual_all_features)
        
        overall_error_percentage = (overall_mae_org/accumulated_y_true.mean())*100

        # Create a dictionary for overall future metrics
        overall_future_metrics  ={
            "Overall Trial": [trial],
            "Overall Future MSE (org)": [round(overall_mse_org, 5)],
            "Overall Future MAE (org)": [round(overall_mae_org, 5)],
            "Overall Future R2 (org)": [round(overall_r2_org, 5)],
            "Overall Future MSE (org all features)": [round(overall_mse_org_all_features , 5)],
            "Overall Future MAE (org all features)": [round(overall_mae_org_all_features, 5)],
            "Overall Future R2 (org all features)": [round(overall_r2_org_all_features , 5)],
            "Overall Future MSE": [round(overall_mse, 5)],
            "Overall Future MAE": [round(overall_mae, 5)],
            "Overall Future R2": [round(overall_r2, 5)],
            "Overall Future MSE (all features)": [round(overall_mse_all_features, 5)],
            "Overall Future MAE (all features)": [round(overall_mae_all_features, 5)],
            "Overall Future R2 (all features)": [round(overall_r2_all_features, 5)],
            "Overall Future Error Percentage": [round(overall_error_percentage, 3)]
        }
        all_overall_future_metrics.append(overall_future_metrics)
        # Convert each dict in the list to a DataFrame
        df_list = [pd.DataFrame(data=d) for d in all_overall_future_metrics]

        # Concatenate the DataFrames
        all_overall_future_metrics_df = pd.concat(df_list, axis=0)
        print(all_overall_future_metrics_df)

        if save_directory:
            all_overall_future_metrics_df.to_csv(f'{save_directory}/{trial}_all_overall_future_metrics.csv', index=True)

        # # Convert dictionary into DataFrame and append it to final results dataframe
        if overall_future_plot:

            combined_predictions = np.concatenate((accumulated_y_pred, future_predictions_target))
            plt.figure(figsize=(15,8))
            plt.plot(np.arange(len(accumulated_y_true)),
                    accumulated_y_true, label='Actual')
            plt.plot(np.arange(len(combined_predictions)),
                    combined_predictions, label='Predicted')
            plt.xlabel('Time Step')
            plt.ylabel('Value')
            plt.title(f'Overall Actual and Predicted Values (Trial {trial+1})')
            plt.legend()

            # plt.savefig(f"overall_predictions_plot_trial_{trial+1}.png")
            if save_directory:
                save_path=os.path.join(save_directory,
                                    f"overall_predictions_plot_trial_{trial+1}.png")
                plt.savefig(save_path)
            plt.show()

        # Add the resulting model to the "top models" list (sorted by Test MSE)
        top_models.append((trial, params, train_mse, train_mae, train_r2, test_mse, test_mae, test_r2))
        top_models.sort(key=lambda x: x[6])
        if len(top_models) > n_top_models:
            top_models.pop()
            
        end = timeit.default_timer()
        # Calculate and print duration
        duration = end - start
        print(f"Execution Time of Trial {trial + 1} of {n_trials} is: {duration} seconds")        
          
    return results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df



In [ ]:
# Chip Maker

import yfinance as yf

start_date = '2018-01-01'
end_date = '2023-09-26'
symbol = 'TSM'
TSM = yf.download(symbol, start=start_date, end=end_date)
TSM = TSM.drop('Adj Close', axis=1)
symbol = 'INTC'
INTC = yf.download(symbol, start=start_date, end=end_date)
INTC = INTC.drop('Adj Close', axis=1)
symbol = 'ASML'
ASML = yf.download(symbol, start=start_date, end=end_date)
ASML = ASML.drop('Adj Close', axis=1)
symbol = 'MU'
MU = yf.download(symbol, start=start_date, end=end_date)
MU = MU.drop('Adj Close', axis=1)
symbol = 'NVDA'
NVDA = yf.download(symbol, start=start_date, end=end_date)
NVDA = NVDA.drop('Adj Close', axis=1)
symbol = 'AMD'
AMD = yf.download(symbol, start=start_date, end=end_date)
AMD = AMD.drop('Adj Close', axis=1)
symbol = 'QCOM'
QCOM = yf.download(symbol, start=start_date, end=end_date)
QCOM = QCOM.drop('Adj Close', axis=1)
symbol = 'SNPS'
SNPS = yf.download(symbol, start=start_date, end=end_date)
SNPS = SNPS.drop('Adj Close', axis=1)
symbol = 'MRVL'
MRVL = yf.download(symbol, start=start_date, end=end_date)
MRVL = MRVL.drop('Adj Close', axis=1)
symbol = '^IXIC'
IXIC = yf.download(symbol, start=start_date, end=end_date)
IXIC = IXIC.drop('Adj Close', axis=1)

# information technology

symbol = 'AAPL'
AAPL = yf.download(symbol, start=start_date, end=end_date)
AAPL = AAPL.drop('Adj Close', axis=1)
symbol = 'MSFT'
MSFT = yf.download(symbol, start=start_date, end=end_date)
MSFT = MSFT.drop('Adj Close', axis=1)
symbol = 'TSLA'
TSLA = yf.download(symbol, start=start_date, end=end_date)
TSLA = TSLA.drop('Adj Close', axis=1)
symbol = 'GOOGL'
GOOGL = yf.download(symbol, start=start_date, end=end_date)
GOOGL = GOOGL.drop('Adj Close', axis=1)
symbol = 'GOOG'
GOOG = yf.download(symbol, start=start_date, end=end_date)
GOOG = GOOG.drop('Adj Close', axis=1)
symbol = 'AMZN'
AMZN = yf.download(symbol, start=start_date, end=end_date)
AMZN = AMZN.drop('Adj Close', axis=1)
symbol = 'META'
META = yf.download(symbol, start=start_date, end=end_date)
META = META.drop('Adj Close', axis=1)
symbol = 'AMD'
AMD = yf.download(symbol, start=start_date, end=end_date)
AMD = AMD.drop('Adj Close', axis=1)
symbol = 'ASML'
ASML = yf.download(symbol, start=start_date, end=end_date)
ASML = ASML.drop('Adj Close', axis=1)
symbol = 'NVDA'
NVDA = yf.download(symbol, start=start_date, end=end_date)
NVDA = NVDA.drop('Adj Close', axis=1)
symbol = 'IBM'
IBM = yf.download(symbol, start=start_date, end=end_date)
IBM = IBM.drop('Adj Close', axis=1)
symbol = 'NFLX'
NFLX = yf.download(symbol, start=start_date, end=end_date)
NFLX = NFLX.drop('Adj Close', axis=1)

# Consumer


symbol = 'WMT'
WMT = yf.download(symbol, start=start_date, end=end_date)
WMT = WMT.drop('Adj Close', axis=1)
symbol = 'TGT'
TGT = yf.download(symbol, start=start_date, end=end_date)
TGT = TGT.drop('Adj Close', axis=1)
symbol = 'COST'
COST = yf.download(symbol, start=start_date, end=end_date)
COST = COST.drop('Adj Close', axis=1)
symbol = 'HD'
HD = yf.download(symbol, start=start_date, end=end_date)
HD = HD.drop('Adj Close', axis=1)
symbol = 'LOW'
LOW = yf.download(symbol, start=start_date, end=end_date)
LOW = LOW.drop('Adj Close', axis=1)

symbol = 'PG'
PG = yf.download(symbol, start=start_date, end=end_date)
PG = PG.drop('Adj Close', axis=1)
symbol = 'JNJ'
JNJ = yf.download(symbol, start=start_date, end=end_date)
JNJ = JNJ.drop('Adj Close', axis=1)
symbol = 'PFE'
PFE = yf.download(symbol, start=start_date, end=end_date)
PFE = PFE.drop('Adj Close', axis=1)
symbol = 'CVS'
CVS = yf.download(symbol, start=start_date, end=end_date)
CVS = CVS.drop('Adj Close', axis=1)

symbol = 'KO'
KO = yf.download(symbol, start=start_date, end=end_date)
KO = KO.drop('Adj Close', axis=1)
symbol = 'PEP'
PEP = yf.download(symbol, start=start_date, end=end_date)
PEP = PEP.drop('Adj Close', axis=1)

symbol = 'NKE'
NKE = yf.download(symbol, start=start_date, end=end_date)
NKE = NKE.drop('Adj Close', axis=1)
symbol = 'MCD'
MCD = yf.download(symbol, start=start_date, end=end_date)
MCD = MCD.drop('Adj Close', axis=1)
symbol = 'SBUX'
SBUX = yf.download(symbol, start=start_date, end=end_date)
SBUX = SBUX.drop('Adj Close', axis=1)

symbol = 'VZ'
VZ = yf.download(symbol, start=start_date, end=end_date)
VZ = VZ.drop('Adj Close', axis=1)
symbol = 'T'
T = yf.download(symbol, start=start_date, end=end_date)
T = T.drop('Adj Close', axis=1)
symbol = 'FOX'
FOX = yf.download(symbol, start=start_date, end=end_date)
FOX = FOX.drop('Adj Close', axis=1)
symbol = 'WBD'
WBD = yf.download(symbol, start=start_date, end=end_date)
WBD = WBD.drop('Adj Close', axis=1)
symbol = 'DIS'
DIS = yf.download(symbol, start=start_date, end=end_date)
DIS = DIS.drop('Adj Close', axis=1)

symbol = 'UPS'
UPS = yf.download(symbol, start=start_date, end=end_date)
UPS = UPS.drop('Adj Close', axis=1)
symbol = 'FDX'
FDX = yf.download(symbol, start=start_date, end=end_date)
FDX = FDX.drop('Adj Close', axis=1)
symbol = 'DAL'
DAL = yf.download(symbol, start=start_date, end=end_date)
DAL = DAL.drop('Adj Close', axis=1)
symbol = 'AAL'
AAL = yf.download(symbol, start=start_date, end=end_date)
AAL = AAL.drop('Adj Close', axis=1)
symbol = 'XOM'
XOM = yf.download(symbol, start=start_date, end=end_date)
XOM = XOM.drop('Adj Close', axis=1)
symbol = 'CVX'
CVX = yf.download(symbol, start=start_date, end=end_date)
CVX = CVX.drop('Adj Close', axis=1)

symbol = 'BAC'
BAC = yf.download(symbol, start=start_date, end=end_date)
BAC = BAC.drop('Adj Close', axis=1)
symbol = 'JPM'
JPM = yf.download(symbol, start=start_date, end=end_date)
JPM = JPM.drop('Adj Close', axis=1)
symbol = 'MA'
MA = yf.download(symbol, start=start_date, end=end_date)
MA = MA.drop('Adj Close', axis=1)
symbol = 'V'
V = yf.download(symbol, start=start_date, end=end_date)
V = V.drop('Adj Close', axis=1)
symbol = 'SPG'
SPG = yf.download(symbol, start=start_date, end=end_date)
SPG = SPG.drop('Adj Close', axis=1)
symbol = 'VNO'
VNO = yf.download(symbol, start=start_date, end=end_date)
VNO = VNO.drop('Adj Close', axis=1)

symbol = 'MMM'
MMM = yf.download(symbol, start=start_date, end=end_date)
MMM = MMM.drop('Adj Close', axis=1)
symbol = 'GE'
GE = yf.download(symbol, start=start_date, end=end_date)
GE = GE.drop('Adj Close', axis=1)
symbol = 'F'
F = yf.download(symbol, start=start_date, end=end_date)
F = F.drop('Adj Close', axis=1)
symbol = 'GM'
GM = yf.download(symbol, start=start_date, end=end_date)
GM = GM.drop('Adj Close', axis=1)
symbol = 'HON'
HON = yf.download(symbol, start=start_date, end=end_date)
HON = HON.drop('Adj Close', axis=1)
symbol = 'LMT'
LMT = yf.download(symbol, start=start_date, end=end_date)
LMT = LMT.drop('Adj Close', axis=1)


# Futures



symbol = 'ES=F'
ESF = yf.download(symbol, start=start_date, end=end_date)
ESF = ESF.drop('Adj Close', axis=1)
symbol = 'YM=F'
YMF = yf.download(symbol, start=start_date, end=end_date)
YMF = YMF.drop('Adj Close', axis=1)
symbol = 'NQ=F'
NQF = yf.download(symbol, start=start_date, end=end_date)
NQF = NQF.drop('Adj Close', axis=1)
symbol = 'RTY=F'
RTYF = yf.download(symbol, start=start_date, end=end_date)
RTYF = RTYF.drop('Adj Close', axis=1)
symbol = 'ZB=F'
ZBF = yf.download(symbol, start=start_date, end=end_date)
ZBF = ZBF.drop('Adj Close', axis=1)
symbol = 'ZN=F'
ZNF = yf.download(symbol, start=start_date, end=end_date)
ZNF = ZNF.drop('Adj Close', axis=1)
symbol = 'ZF=F'
ZFF = yf.download(symbol, start=start_date, end=end_date)
ZFF = ZFF.drop('Adj Close', axis=1)
symbol = 'ZT=F'
ZTF = yf.download(symbol, start=start_date, end=end_date)
ZTF = ZTF.drop('Adj Close', axis=1)
symbol = 'GC=F'
GCF = yf.download(symbol, start=start_date, end=end_date)
GCF = GCF.drop('Adj Close', axis=1)
symbol = 'HG=F'
HGF = yf.download(symbol, start=start_date, end=end_date)
HGF = HGF.drop('Adj Close', axis=1)
symbol = 'SI=F'
SIF = yf.download(symbol, start=start_date, end=end_date)
SIF = SIF.drop('Adj Close', axis=1)
symbol = 'PL=F'
PLF = yf.download(symbol, start=start_date, end=end_date)
PLF = PLF.drop('Adj Close', axis=1)
CLF = yf.download(symbol, start=start_date, end=end_date)
CLF = CLF.drop('Adj Close', axis=1)
symbol = 'NG=F'
NGF = yf.download(symbol, start=start_date, end=end_date)
NGF = NGF.drop('Adj Close', axis=1)
symbol = 'BZ=F'
BZF = yf.download(symbol, start=start_date, end=end_date)
BZF = BZF.drop('Adj Close', axis=1)
symbol = 'ZC=F'
ZCF = yf.download(symbol, start=start_date, end=end_date)
ZCF = ZCF.drop('Adj Close', axis=1)
symbol = 'ZO=F'
ZOF = yf.download(symbol, start=start_date, end=end_date)
ZOF = ZOF.drop('Adj Close', axis=1)
symbol = 'KE=F'
KEF = yf.download(symbol, start=start_date, end=end_date)
KEF = KEF.drop('Adj Close', axis=1)
symbol = 'ZR=F'
ZRF = yf.download(symbol, start=start_date, end=end_date)
ZRF = ZRF.drop('Adj Close', axis=1)
symbol = 'ZM=F'
ZMF = yf.download(symbol, start=start_date, end=end_date)
ZMF = ZMF.drop('Adj Close', axis=1)
symbol = 'ZL=F'
ZLF = yf.download(symbol, start=start_date, end=end_date)
ZLF = ZLF.drop('Adj Close', axis=1)
symbol = 'ZS=F'
ZSF = yf.download(symbol, start=start_date, end=end_date)
ZSF = ZSF.drop('Adj Close', axis=1)
symbol = 'GF=F'
GFF = yf.download(symbol, start=start_date, end=end_date)
GFF = GFF.drop('Adj Close', axis=1)
symbol = 'HE=F'
HEF = yf.download(symbol, start=start_date, end=end_date)
HEF = HEF.drop('Adj Close', axis=1)
symbol = 'HO=F'
HOF = yf.download(symbol, start=start_date, end=end_date)
HOF = HOF.drop('Adj Close', axis=1)
symbol = 'LE=F'
LFF = yf.download(symbol, start=start_date, end=end_date)
LFF = LFF.drop('Adj Close', axis=1)
symbol = 'CC=F'
CCF = yf.download(symbol, start=start_date, end=end_date)
CCF = CCF.drop('Adj Close', axis=1)
symbol = 'KC=F'
KCF = yf.download(symbol, start=start_date, end=end_date)
KCF = KCF.drop('Adj Close', axis=1)
symbol = 'CT=F'
CTF = yf.download(symbol, start=start_date, end=end_date)
CTF = CTF.drop('Adj Close', axis=1)
symbol = 'OJ=F'
OJF = yf.download(symbol, start=start_date, end=end_date)
OJF = OJF.drop('Adj Close', axis=1)
symbol = 'SB=F'
SBF = yf.download(symbol, start=start_date, end=end_date)
SBF = SBF.drop('Adj Close', axis=1)


# EFTs

symbol = 'KBA'
KBA = yf.download(symbol, start=start_date, end=end_date)
KBA = KBA.drop('Adj Close', axis=1)
symbol = 'CHIQ'
CHIQ = yf.download(symbol, start=start_date, end=end_date)
CHIQ = CHIQ.drop('Adj Close', axis=1)
symbol = 'CNTX'
CNTX = yf.download(symbol, start=start_date, end=end_date)
CNTX = CNTX.drop('Adj Close', axis=1)
symbol = 'CHIS'
CHIS = yf.download(symbol, start=start_date, end=end_date)
CHIS = CHIS.drop('Adj Close', axis=1)
symbol = 'CNYA'
CNYA = yf.download(symbol, start=start_date, end=end_date)
CNYA = CNYA.drop('Adj Close', axis=1)
symbol = 'ASHX'
ASHX = yf.download(symbol, start=start_date, end=end_date)
ASHX = ASHX.drop('Adj Close', axis=1)
symbol = 'KFYP'
KFYP = yf.download(symbol, start=start_date, end=end_date)
KFYP = KFYP.drop('Adj Close', axis=1)
symbol = 'KGRN'
KGRN = yf.download(symbol, start=start_date, end=end_date)
KGRN = KGRN.drop('Adj Close', axis=1)
symbol = 'THD'
THD = yf.download(symbol, start=start_date, end=end_date)
THD = THD.drop('Adj Close', axis=1)
symbol = 'BBAX'
BBAX = yf.download(symbol, start=start_date, end=end_date)
BBAX = BBAX.drop('Adj Close', axis=1)
symbol = 'FEMS'
FEMS = yf.download(symbol, start=start_date, end=end_date)
FEMS = FEMS.drop('Adj Close', axis=1)
symbol = 'EZA'
EZA = yf.download(symbol, start=start_date, end=end_date)
EZA = EZA.drop('Adj Close', axis=1)
symbol = 'XSD'
XSD = yf.download(symbol, start=start_date, end=end_date)
XSD = XSD.drop('Adj Close', axis=1)
symbol = 'EYLD'
EYLD = yf.download(symbol, start=start_date, end=end_date)
EYLD = EYLD.drop('Adj Close', axis=1)
symbol = 'FNDE'
FNDE = yf.download(symbol, start=start_date, end=end_date)
FNDE = FNDE.drop('Adj Close', axis=1)
symbol = 'SPEM'
SPEM = yf.download(symbol, start=start_date, end=end_date)
SPEM = SPEM.drop('Adj Close', axis=1)
symbol = 'DXJS'
DXJS = yf.download(symbol, start=start_date, end=end_date)
DXJS = DXJS.drop('Adj Close', axis=1)
symbol = 'KURE'
KURE = yf.download(symbol, start=start_date, end=end_date)
KURE = KURE.drop('Adj Close', axis=1)
symbol = 'EWX'
EWX = yf.download(symbol, start=start_date, end=end_date)
EWX = EWX.drop('Adj Close', axis=1)
symbol = 'FLJH'
FLJH = yf.download(symbol, start=start_date, end=end_date)
FLJH = FLJH.drop('Adj Close', axis=1)
symbol = 'CQQQ'
CQQQ = yf.download(symbol, start=start_date, end=end_date)
CQQQ = CQQQ.drop('Adj Close', axis=1)
symbol = 'CHIE'
CHIE = yf.download(symbol, start=start_date, end=end_date)
CHIE = CHIE.drop('Adj Close', axis=1)
symbol = 'MFEM'
MFEM = yf.download(symbol, start=start_date, end=end_date)
MFEM = MFEM.drop('Adj Close', axis=1)
symbol = 'DGS'
DGS = yf.download(symbol, start=start_date, end=end_date)
DGS = DGS.drop('Adj Close', axis=1)
symbol = 'HEEM'
HEEM = yf.download(symbol, start=start_date, end=end_date)
HEEM = HEEM.drop('Adj Close', axis=1)


# World Composite Index


symbol = '^HSI'
HSI = yf.download(symbol, start=start_date, end=end_date)
HSI = HSI.drop('Adj Close', axis=1)
symbol = '000001.SS'
SSE = yf.download(symbol, start=start_date, end=end_date)
SSE = SSE.drop('Adj Close', axis=1)
symbol = '^N225'
N225 = yf.download(symbol, start=start_date, end=end_date)
N225 = N225.drop('Adj Close', axis=1)
symbol = '^KS11'
KS11 = yf.download(symbol, start=start_date, end=end_date)
KS11 = KS11.drop('Adj Close', axis=1)
symbol = '^BSESN'
BSESN = yf.download(symbol, start=start_date, end=end_date)
BSESN = BSESN.drop('Adj Close', axis=1)
symbol = '^MXX'
MXX = yf.download(symbol, start=start_date, end=end_date)
MXX = MXX.drop('Adj Close', axis=1)
symbol = '^TNX'
TNX = yf.download(symbol, start=start_date, end=end_date)
TNX = TNX.drop('Adj Close', axis=1)
symbol = '^VIX'
VIX = yf.download(symbol, start=start_date, end=end_date)
VIX = VIX.drop('Adj Close', axis=1)
symbol = '^BVSP'
BVSP = yf.download(symbol, start=start_date, end=end_date)
BVSP = BVSP.drop('Adj Close', axis=1)
symbol = '^IXIC'
IXIC = yf.download(symbol, start=start_date, end=end_date)
IXIC = IXIC.drop('Adj Close', axis=1)
symbol = '^GSPTSE'
GSPTSE = yf.download(symbol, start=start_date, end=end_date)
GSPTSE = GSPTSE.drop('Adj Close', axis=1)
symbol = '^DJI'
DJI = yf.download(symbol, start=start_date, end=end_date)
DJI = DJI.drop('Adj Close', axis=1)
symbol = '^FCHI'
FCHI = yf.download(symbol, start=start_date, end=end_date)
FCHI = FCHI.drop('Adj Close', axis=1)
symbol = '^GDAXI'
GDAXI = yf.download(symbol, start=start_date, end=end_date)
GDAXI = GDAXI.drop('Adj Close', axis=1)
symbol = '^FTSE'
FTSE = yf.download(symbol, start=start_date, end=end_date)
FTSE = FTSE.drop('Adj Close', axis=1)
symbol = '^IBEX'
IBEX = yf.download(symbol, start=start_date, end=end_date)
IBEX = IBEX.drop('Adj Close', axis=1)
symbol = '^N100'
N100 = yf.download(symbol, start=start_date, end=end_date)
N100 = N100.drop('Adj Close', axis=1)
symbol = '^GSPC'
GSPC = yf.download(symbol, start=start_date, end=end_date)
GSPC = GSPC.drop('Adj Close', axis=1)
symbol = '^RUT'
RUT = yf.download(symbol, start=start_date, end=end_date)
RUT = RUT.drop('Adj Close', axis=1)
symbol = '^NYA'
NYA = yf.download(symbol, start=start_date, end=end_date)
NYA = NYA.drop('Adj Close', axis=1)
symbol = '^STI'
STI = yf.download(symbol, start=start_date, end=end_date)
STI = STI.drop('Adj Close', axis=1)
symbol = '^AXJO'
AXJO = yf.download(symbol, start=start_date, end=end_date)
AXJO = AXJO.drop('Adj Close', axis=1)



In [ ]:
# Chip Maker

import yfinance as yf

start_date  = '2023-01-01'
end_date  = '2023-09-26'
symbol  = 'TSM'
TSM1  = yf.download(symbol, start=start_date, end=end_date)
TSM1  = TSM1.drop('Adj Close', axis=1)
symbol  = 'INTC'
INTC1  = yf.download(symbol, start=start_date, end=end_date)
INTC1  = INTC1.drop('Adj Close', axis=1)
symbol  = 'ASML'
ASML1  = yf.download(symbol, start=start_date, end=end_date)
ASML1  = ASML1.drop('Adj Close', axis=1)
symbol  = 'MU'
MU1  = yf.download(symbol, start=start_date, end=end_date)
MU1  = MU1.drop('Adj Close', axis=1)
symbol  = 'NVDA'
NVDA1  = yf.download(symbol, start=start_date, end=end_date)
NVDA1  = NVDA1.drop('Adj Close', axis=1)
symbol  = 'AMD'
AMD1  = yf.download(symbol, start=start_date, end=end_date)
AMD1  = AMD1.drop('Adj Close', axis=1)
symbol  = 'QCOM'
QCOM1  = yf.download(symbol, start=start_date, end=end_date)
QCOM1  = QCOM1.drop('Adj Close', axis=1)
symbol  = 'SNPS'
SNPS1  = yf.download(symbol, start=start_date, end=end_date)
SNPS1  = SNPS1.drop('Adj Close', axis=1)
symbol  = 'MRVL'
MRVL1  = yf.download(symbol, start=start_date, end=end_date)
MRVL1  = MRVL1.drop('Adj Close', axis=1)
symbol  = '^IXIC'
IXIC1  = yf.download(symbol, start=start_date, end=end_date)
IXIC1  = IXIC1.drop('Adj Close', axis=1)

# information technology

symbol  = 'AAPL'
AAPL1  = yf.download(symbol, start=start_date, end=end_date)
AAPL1  = AAPL1.drop('Adj Close', axis=1)
symbol  = 'MSFT'
MSFT1  = yf.download(symbol, start=start_date, end=end_date)
MSFT1  = MSFT1.drop('Adj Close', axis=1)
symbol  = 'TSLA'
TSLA1  = yf.download(symbol, start=start_date, end=end_date)
TSLA1  = TSLA1.drop('Adj Close', axis=1)
symbol  = 'GOOGL'
GOOGL1  = yf.download(symbol, start=start_date, end=end_date)
GOOGL1  = GOOGL1.drop('Adj Close', axis=1)
symbol  = 'GOOG'
GOOG1  = yf.download(symbol, start=start_date, end=end_date)
GOOG1  = GOOG1.drop('Adj Close', axis=1)
symbol  = 'AMZN'
AMZN1  = yf.download(symbol, start=start_date, end=end_date)
AMZN1  = AMZN1.drop('Adj Close', axis=1)
symbol  = 'META'
META1  = yf.download(symbol, start=start_date, end=end_date)
META1  = META1.drop('Adj Close', axis=1)
symbol  = 'AMD'
AMD1  = yf.download(symbol, start=start_date, end=end_date)
AMD1  = AMD1.drop('Adj Close', axis=1)
symbol  = 'ASML'
ASML1  = yf.download(symbol, start=start_date, end=end_date)
ASML1  = ASML1.drop('Adj Close', axis=1)
symbol  = 'NVDA'
NVDA1  = yf.download(symbol, start=start_date, end=end_date)
NVDA1  = NVDA1.drop('Adj Close', axis=1)
symbol  = 'IBM'
IBM1  = yf.download(symbol, start=start_date, end=end_date)
IBM1  = IBM1.drop('Adj Close', axis=1)
symbol  = 'NFLX'
NFLX1  = yf.download(symbol, start=start_date, end=end_date)
NFLX1  = NFLX1.drop('Adj Close', axis=1)

# Consumer


symbol  = 'WMT'
WMT1  = yf.download(symbol, start=start_date, end=end_date)
WMT1  = WMT1.drop('Adj Close', axis=1)
symbol  = 'TGT'
TGT1  = yf.download(symbol, start=start_date, end=end_date)
TGT1  = TGT1.drop('Adj Close', axis=1)
symbol  = 'COST'
COST1  = yf.download(symbol, start=start_date, end=end_date)
COST1  = COST1.drop('Adj Close', axis=1)
symbol  = 'HD'
HD1  = yf.download(symbol, start=start_date, end=end_date)
HD1  = HD1.drop('Adj Close', axis=1)
symbol  = 'LOW'
LOW1  = yf.download(symbol, start=start_date, end=end_date)
LOW1  = LOW1.drop('Adj Close', axis=1)

symbol  = 'PG'
PG1  = yf.download(symbol, start=start_date, end=end_date)
PG1  = PG1.drop('Adj Close', axis=1)
symbol  = 'JNJ'
JNJ1  = yf.download(symbol, start=start_date, end=end_date)
JNJ1  = JNJ1.drop('Adj Close', axis=1)
symbol  = 'PFE'
PFE1  = yf.download(symbol, start=start_date, end=end_date)
PFE1  = PFE1.drop('Adj Close', axis=1)
symbol  = 'CVS'
CVS1  = yf.download(symbol, start=start_date, end=end_date)
CVS1  = CVS1.drop('Adj Close', axis=1)

symbol  = 'KO'
KO1  = yf.download(symbol, start=start_date, end=end_date)
KO1  = KO1.drop('Adj Close', axis=1)
symbol  = 'PEP'
PEP1  = yf.download(symbol, start=start_date, end=end_date)
PEP1  = PEP1.drop('Adj Close', axis=1)

symbol  = 'NKE'
NKE1  = yf.download(symbol, start=start_date, end=end_date)
NKE1  = NKE1.drop('Adj Close', axis=1)
symbol  = 'MCD'
MCD1  = yf.download(symbol, start=start_date, end=end_date)
MCD1  = MCD1.drop('Adj Close', axis=1)
symbol  = 'SBUX'
SBUX1  = yf.download(symbol, start=start_date, end=end_date)
SBUX1  = SBUX1.drop('Adj Close', axis=1)

symbol  = 'VZ'
VZ1  = yf.download(symbol, start=start_date, end=end_date)
VZ1  = VZ1.drop('Adj Close', axis=1)
symbol  = 'T'
T1  = yf.download(symbol, start=start_date, end=end_date)
T1  = T1.drop('Adj Close', axis=1)
symbol  = 'FOX'
FOX1  = yf.download(symbol, start=start_date, end=end_date)
FOX1  = FOX1.drop('Adj Close', axis=1)
symbol  = 'WBD'
WBD1  = yf.download(symbol, start=start_date, end=end_date)
WBD1  = WBD1.drop('Adj Close', axis=1)
symbol  = 'DIS'
DIS1  = yf.download(symbol, start=start_date, end=end_date)
DIS1  = DIS1.drop('Adj Close', axis=1)

symbol  = 'UPS'
UPS1  = yf.download(symbol, start=start_date, end=end_date)
UPS1  = UPS1.drop('Adj Close', axis=1)
symbol  = 'FDX'
FDX1  = yf.download(symbol, start=start_date, end=end_date)
FDX1  = FDX1.drop('Adj Close', axis=1)
symbol  = 'DAL'
DAL1  = yf.download(symbol, start=start_date, end=end_date)
DAL1  = DAL1.drop('Adj Close', axis=1)
symbol  = 'AAL'
AAL1  = yf.download(symbol, start=start_date, end=end_date)
AAL1  = AAL1.drop('Adj Close', axis=1)
symbol  = 'XOM'
XOM1  = yf.download(symbol, start=start_date, end=end_date)
XOM1  = XOM1.drop('Adj Close', axis=1)
symbol  = 'CVX'
CVX1  = yf.download(symbol, start=start_date, end=end_date)
CVX1  = CVX1.drop('Adj Close', axis=1)

symbol  = 'BAC'
BAC1  = yf.download(symbol, start=start_date, end=end_date)
BAC1  = BAC1.drop('Adj Close', axis=1)
symbol  = 'JPM'
JPM1  = yf.download(symbol, start=start_date, end=end_date)
JPM1  = JPM1.drop('Adj Close', axis=1)
symbol  = 'MA'
MA1  = yf.download(symbol, start=start_date, end=end_date)
MA1  = MA1.drop('Adj Close', axis=1)
symbol  = 'V'
V1  = yf.download(symbol, start=start_date, end=end_date)
V1  = V1.drop('Adj Close', axis=1)
symbol  = 'SPG'
SPG1  = yf.download(symbol, start=start_date, end=end_date)
SPG1  = SPG1.drop('Adj Close', axis=1)
symbol  = 'VNO'
VNO1  = yf.download(symbol, start=start_date, end=end_date)
VNO1  = VNO1.drop('Adj Close', axis=1)

symbol  = 'MMM'
MMM1  = yf.download(symbol, start=start_date, end=end_date)
MMM1  = MMM1.drop('Adj Close', axis=1)
symbol  = 'GE'
GE1  = yf.download(symbol, start=start_date, end=end_date)
GE1  = GE1.drop('Adj Close', axis=1)
symbol  = 'F'
F1  = yf.download(symbol, start=start_date, end=end_date)
F1  = F1.drop('Adj Close', axis=1)
symbol  = 'GM'
GM1  = yf.download(symbol, start=start_date, end=end_date)
GM1  = GM1.drop('Adj Close', axis=1)
symbol  = 'HON'
HON1  = yf.download(symbol, start=start_date, end=end_date)
HON1  = HON1.drop('Adj Close', axis=1)
symbol  = 'LMT'
LMT1  = yf.download(symbol, start=start_date, end=end_date)
LMT1  = LMT1.drop('Adj Close', axis=1)


# Futures



symbol  = 'ES=F'
ESF1  = yf.download(symbol, start=start_date, end=end_date)
ESF1  = ESF1.drop('Adj Close', axis=1)
symbol  = 'YM=F'
YMF1  = yf.download(symbol, start=start_date, end=end_date)
YMF1  = YMF1.drop('Adj Close', axis=1)
symbol  = 'NQ=F'
NQF1  = yf.download(symbol, start=start_date, end=end_date)
NQF1  = NQF1.drop('Adj Close', axis=1)
symbol  = 'RTY=F'
RTYF1  = yf.download(symbol, start=start_date, end=end_date)
RTYF1  = RTYF1.drop('Adj Close', axis=1)
symbol  = 'ZB=F'
ZBF1  = yf.download(symbol, start=start_date, end=end_date)
ZBF1  = ZBF1.drop('Adj Close', axis=1)
symbol  = 'ZN=F'
ZNF1  = yf.download(symbol, start=start_date, end=end_date)
ZNF1  = ZNF1.drop('Adj Close', axis=1)
symbol  = 'ZF=F'
ZFF1  = yf.download(symbol, start=start_date, end=end_date)
ZFF1  = ZFF1.drop('Adj Close', axis=1)
symbol  = 'ZT=F'
ZTF1  = yf.download(symbol, start=start_date, end=end_date)
ZTF1  = ZTF1.drop('Adj Close', axis=1)
symbol  = 'GC=F'
GCF1  = yf.download(symbol, start=start_date, end=end_date)
GCF1  = GCF1.drop('Adj Close', axis=1)
symbol  = 'HG=F'
HGF1  = yf.download(symbol, start=start_date, end=end_date)
HGF1  = HGF1.drop('Adj Close', axis=1)
symbol  = 'SI=F'
SIF1  = yf.download(symbol, start=start_date, end=end_date)
SIF1  = SIF1.drop('Adj Close', axis=1)
symbol  = 'PL=F'
PLF1  = yf.download(symbol, start=start_date, end=end_date)
PLF1  = PLF1.drop('Adj Close', axis=1)
CLF1  = yf.download(symbol, start=start_date, end=end_date)
CLF1  = CLF1.drop('Adj Close', axis=1)
symbol  = 'NG=F'
NGF1  = yf.download(symbol, start=start_date, end=end_date)
NGF1  = NGF1.drop('Adj Close', axis=1)
symbol  = 'BZ=F'
BZF1  = yf.download(symbol, start=start_date, end=end_date)
BZF1  = BZF1.drop('Adj Close', axis=1)
symbol  = 'ZC=F'
ZCF1  = yf.download(symbol, start=start_date, end=end_date)
ZCF1  = ZCF1.drop('Adj Close', axis=1)
symbol  = 'ZO=F'
ZOF1  = yf.download(symbol, start=start_date, end=end_date)
ZOF1  = ZOF1.drop('Adj Close', axis=1)
symbol  = 'KE=F'
KEF1  = yf.download(symbol, start=start_date, end=end_date)
KEF1  = KEF1.drop('Adj Close', axis=1)
symbol  = 'ZR=F'
ZRF1  = yf.download(symbol, start=start_date, end=end_date)
ZRF1  = ZRF1.drop('Adj Close', axis=1)
symbol  = 'ZM=F'
ZMF1  = yf.download(symbol, start=start_date, end=end_date)
ZMF1  = ZMF1.drop('Adj Close', axis=1)
symbol  = 'ZL=F'
ZLF1  = yf.download(symbol, start=start_date, end=end_date)
ZLF1  = ZLF1.drop('Adj Close', axis=1)
symbol  = 'ZS=F'
ZSF1  = yf.download(symbol, start=start_date, end=end_date)
ZSF1  = ZSF1.drop('Adj Close', axis=1)
symbol  = 'GF=F'
GFF1  = yf.download(symbol, start=start_date, end=end_date)
GFF1  = GFF1.drop('Adj Close', axis=1)
symbol  = 'HE=F'
HEF1  = yf.download(symbol, start=start_date, end=end_date)
HEF1  = HEF1.drop('Adj Close', axis=1)
symbol  = 'HO=F'
HOF1  = yf.download(symbol, start=start_date, end=end_date)
HOF1  = HOF1.drop('Adj Close', axis=1)
symbol  = 'LE=F'
LFF1  = yf.download(symbol, start=start_date, end=end_date)
LFF1  = LFF1.drop('Adj Close', axis=1)
symbol  = 'CC=F'
CCF1  = yf.download(symbol, start=start_date, end=end_date)
CCF1  = CCF1.drop('Adj Close', axis=1)
symbol  = 'KC=F'
KCF1  = yf.download(symbol, start=start_date, end=end_date)
KCF1  = KCF1.drop('Adj Close', axis=1)
symbol  = 'CT=F'
CTF1  = yf.download(symbol, start=start_date, end=end_date)
CTF1  = CTF1.drop('Adj Close', axis=1)
symbol  = 'OJ=F'
OJF1  = yf.download(symbol, start=start_date, end=end_date)
OJF1  = OJF1.drop('Adj Close', axis=1)
symbol  = 'SB=F'
SBF1  = yf.download(symbol, start=start_date, end=end_date)
SBF1  = SBF1.drop('Adj Close', axis=1)


# EFTs

symbol  = 'KBA'
KBA1  = yf.download(symbol, start=start_date, end=end_date)
KBA1  = KBA1.drop('Adj Close', axis=1)
symbol  = 'CHIQ'
CHIQ1  = yf.download(symbol, start=start_date, end=end_date)
CHIQ1  = CHIQ1.drop('Adj Close', axis=1)
symbol  = 'CNTX'
CNTX1  = yf.download(symbol, start=start_date, end=end_date)
CNTX1  = CNTX1.drop('Adj Close', axis=1)
symbol  = 'CHIS'
CHIS1  = yf.download(symbol, start=start_date, end=end_date)
CHIS1  = CHIS1.drop('Adj Close', axis=1)
symbol  = 'CNYA'
CNYA1  = yf.download(symbol, start=start_date, end=end_date)
CNYA1  = CNYA1.drop('Adj Close', axis=1)
symbol  = 'ASHX'
ASHX1  = yf.download(symbol, start=start_date, end=end_date)
ASHX1  = ASHX1.drop('Adj Close', axis=1)
symbol  = 'KFYP'
KFYP1  = yf.download(symbol, start=start_date, end=end_date)
KFYP1  = KFYP1.drop('Adj Close', axis=1)
symbol  = 'KGRN'
KGRN1  = yf.download(symbol, start=start_date, end=end_date)
KGRN1  = KGRN1.drop('Adj Close', axis=1)
symbol  = 'THD'
THD1  = yf.download(symbol, start=start_date, end=end_date)
THD1  = THD1.drop('Adj Close', axis=1)
symbol  = 'BBAX'
BBAX1  = yf.download(symbol, start=start_date, end=end_date)
BBAX1  = BBAX1.drop('Adj Close', axis=1)
symbol  = 'FEMS'
FEMS1  = yf.download(symbol, start=start_date, end=end_date)
FEMS1  = FEMS1.drop('Adj Close', axis=1)
symbol  = 'EZA'
EZA1  = yf.download(symbol, start=start_date, end=end_date)
EZA1  = EZA1.drop('Adj Close', axis=1)
symbol  = 'XSD'
XSD1  = yf.download(symbol, start=start_date, end=end_date)
XSD1  = XSD1.drop('Adj Close', axis=1)
symbol  = 'EYLD'
EYLD1  = yf.download(symbol, start=start_date, end=end_date)
EYLD1  = EYLD1.drop('Adj Close', axis=1)
symbol  = 'FNDE'
FNDE1  = yf.download(symbol, start=start_date, end=end_date)
FNDE1  = FNDE1.drop('Adj Close', axis=1)
symbol  = 'SPEM'
SPEM1  = yf.download(symbol, start=start_date, end=end_date)
SPEM1  = SPEM1.drop('Adj Close', axis=1)
symbol  = 'DXJS'
DXJS1  = yf.download(symbol, start=start_date, end=end_date)
DXJS1  = DXJS1.drop('Adj Close', axis=1)
symbol  = 'KURE'
KURE1  = yf.download(symbol, start=start_date, end=end_date)
KURE1  = KURE1.drop('Adj Close', axis=1)
symbol  = 'EWX'
EWX1  = yf.download(symbol, start=start_date, end=end_date)
EWX1  = EWX1.drop('Adj Close', axis=1)
symbol  = 'FLJH'
FLJH1  = yf.download(symbol, start=start_date, end=end_date)
FLJH1  = FLJH1.drop('Adj Close', axis=1)
symbol  = 'CQQQ'
CQQQ1  = yf.download(symbol, start=start_date, end=end_date)
CQQQ1  = CQQQ1.drop('Adj Close', axis=1)
symbol  = 'CHIE'
CHIE1  = yf.download(symbol, start=start_date, end=end_date)
CHIE1  = CHIE1.drop('Adj Close', axis=1)
symbol  = 'MFEM'
MFEM1  = yf.download(symbol, start=start_date, end=end_date)
MFEM1  = MFEM1.drop('Adj Close', axis=1)
symbol  = 'DGS'
DGS1  = yf.download(symbol, start=start_date, end=end_date)
DGS1  = DGS1.drop('Adj Close', axis=1)
symbol  = 'HEEM'
HEEM1  = yf.download(symbol, start=start_date, end=end_date)
HEEM1  = HEEM1.drop('Adj Close', axis=1)


# World Composite Index


symbol  = '^HSI'
HSI1  = yf.download(symbol, start=start_date, end=end_date)
HSI1  = HSI1.drop('Adj Close', axis=1)
symbol  = '000001.SS'
SSE1  = yf.download(symbol, start=start_date, end=end_date)
SSE1  = SSE1.drop('Adj Close', axis=1)
symbol  = '^N225'
N2251  = yf.download(symbol, start=start_date, end=end_date)
N2251  = N2251.drop('Adj Close', axis=1)
symbol  = '^KS11'
KS111  = yf.download(symbol, start=start_date, end=end_date)
KS111  = KS111.drop('Adj Close', axis=1)
symbol  = '^BSESN'
BSESN1  = yf.download(symbol, start=start_date, end=end_date)
BSESN1  = BSESN1.drop('Adj Close', axis=1)
symbol  = '^MXX'
MXX1  = yf.download(symbol, start=start_date, end=end_date)
MXX1  = MXX1.drop('Adj Close', axis=1)
symbol  = '^TNX'
TNX1  = yf.download(symbol, start=start_date, end=end_date)
TNX1  = TNX1.drop('Adj Close', axis=1)
symbol  = '^VIX'
VIX1  = yf.download(symbol, start=start_date, end=end_date)
VIX1  = VIX1.drop('Adj Close', axis=1)
symbol  = '^BVSP'
BVSP1  = yf.download(symbol, start=start_date, end=end_date)
BVSP1  = BVSP1.drop('Adj Close', axis=1)
symbol  = '^IXIC'
IXIC1  = yf.download(symbol, start=start_date, end=end_date)
IXIC1  = IXIC1.drop('Adj Close', axis=1)
symbol  = '^GSPTSE'
GSPTSE1  = yf.download(symbol, start=start_date, end=end_date)
GSPTSE1  = GSPTSE1.drop('Adj Close', axis=1)
symbol  = '^DJI'
DJI1  = yf.download(symbol, start=start_date, end=end_date)
DJI1  = DJI1.drop('Adj Close', axis=1)
symbol  = '^FCHI'
FCHI1  = yf.download(symbol, start=start_date, end=end_date)
FCHI1  = FCHI1.drop('Adj Close', axis=1)
symbol  = '^GDAXI'
GDAXI1  = yf.download(symbol, start=start_date, end=end_date)
GDAXI1  = GDAXI1.drop('Adj Close', axis=1)
symbol  = '^FTSE'
FTSE1  = yf.download(symbol, start=start_date, end=end_date)
FTSE1  = FTSE1.drop('Adj Close', axis=1)
symbol  = '^IBEX'
IBEX1  = yf.download(symbol, start=start_date, end=end_date)
IBEX1  = IBEX1.drop('Adj Close', axis=1)
symbol  = '^N100'
N1001  = yf.download(symbol, start=start_date, end=end_date)
N1001  = N1001.drop('Adj Close', axis=1)
symbol  = '^GSPC'
GSPC1  = yf.download(symbol, start=start_date, end=end_date)
GSPC1  = GSPC1.drop('Adj Close', axis=1)
symbol  = '^RUT'
RUT1  = yf.download(symbol, start=start_date, end=end_date)
RUT1  = RUT1.drop('Adj Close', axis=1)
symbol  = '^NYA'
NYA1  = yf.download(symbol, start=start_date, end=end_date)
NYA1  = NYA1.drop('Adj Close', axis=1)
symbol  = '^STI'
STI1  = yf.download(symbol, start=start_date, end=end_date)
STI1  = STI1.drop('Adj Close', axis=1)
symbol  = '^AXJO'
AXJO1  = yf.download(symbol, start=start_date, end=end_date)
AXJO1  = AXJO1.drop('Adj Close', axis=1)

In [ ]:
# # Chip Maker

# import yfinance as yf

# start_date = '2018-01-01'
# end_date = '2023-09-26'
# symbol = 'TSM'
# TSM = yf.download(symbol, start=start_date, end=end_date)
# TSM = TSM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'INTC'
# INTC = yf.download(symbol, start=start_date, end=end_date)
# INTC = INTC.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ASML'
# ASML = yf.download(symbol, start=start_date, end=end_date)
# ASML = ASML.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MU'
# MU = yf.download(symbol, start=start_date, end=end_date)
# MU = MU.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'NVDA'
# NVDA = yf.download(symbol, start=start_date, end=end_date)
# NVDA = NVDA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'AMD'
# AMD = yf.download(symbol, start=start_date, end=end_date)
# AMD = AMD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'QCOM'
# QCOM = yf.download(symbol, start=start_date, end=end_date)
# QCOM = QCOM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SNPS'
# SNPS = yf.download(symbol, start=start_date, end=end_date)
# SNPS = SNPS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MRVL'
# MRVL = yf.download(symbol, start=start_date, end=end_date)
# MRVL = MRVL.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^IXIC'
# IXIC = yf.download(symbol, start=start_date, end=end_date)
# IXIC = IXIC.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# # information technology

# symbol = 'AAPL'
# AAPL = yf.download(symbol, start=start_date, end=end_date)
# AAPL = AAPL.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MSFT'
# MSFT = yf.download(symbol, start=start_date, end=end_date)
# MSFT = MSFT.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'TSLA'
# TSLA = yf.download(symbol, start=start_date, end=end_date)
# TSLA = TSLA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GOOGL'
# GOOGL = yf.download(symbol, start=start_date, end=end_date)
# GOOGL = GOOGL.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GOOG'
# GOOG = yf.download(symbol, start=start_date, end=end_date)
# GOOG = GOOG.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'AMZN'
# AMZN = yf.download(symbol, start=start_date, end=end_date)
# AMZN = AMZN.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'META'
# META = yf.download(symbol, start=start_date, end=end_date)
# META = META.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'AMD'
# AMD = yf.download(symbol, start=start_date, end=end_date)
# AMD = AMD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ASML'
# ASML = yf.download(symbol, start=start_date, end=end_date)
# ASML = ASML.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'NVDA'
# NVDA = yf.download(symbol, start=start_date, end=end_date)
# NVDA = NVDA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'IBM'
# IBM = yf.download(symbol, start=start_date, end=end_date)
# IBM = IBM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'NFLX'
# NFLX = yf.download(symbol, start=start_date, end=end_date)
# NFLX = NFLX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# # Consumer


# symbol = 'WMT'
# WMT = yf.download(symbol, start=start_date, end=end_date)
# WMT = WMT.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'TGT'
# TGT = yf.download(symbol, start=start_date, end=end_date)
# TGT = TGT.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'COST'
# COST = yf.download(symbol, start=start_date, end=end_date)
# COST = COST.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HD'
# HD = yf.download(symbol, start=start_date, end=end_date)
# HD = HD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'LOW'
# LOW = yf.download(symbol, start=start_date, end=end_date)
# LOW = LOW.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'PG'
# PG = yf.download(symbol, start=start_date, end=end_date)
# PG = PG.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'JNJ'
# JNJ = yf.download(symbol, start=start_date, end=end_date)
# JNJ = JNJ.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'PFE'
# PFE = yf.download(symbol, start=start_date, end=end_date)
# PFE = PFE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CVS'
# CVS = yf.download(symbol, start=start_date, end=end_date)
# CVS = CVS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'KO'
# KO = yf.download(symbol, start=start_date, end=end_date)
# KO = KO.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'PEP'
# PEP = yf.download(symbol, start=start_date, end=end_date)
# PEP = PEP.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'NKE'
# NKE = yf.download(symbol, start=start_date, end=end_date)
# NKE = NKE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MCD'
# MCD = yf.download(symbol, start=start_date, end=end_date)
# MCD = MCD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SBUX'
# SBUX = yf.download(symbol, start=start_date, end=end_date)
# SBUX = SBUX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'VZ'
# VZ = yf.download(symbol, start=start_date, end=end_date)
# VZ = VZ.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'T'
# T = yf.download(symbol, start=start_date, end=end_date)
# T = T.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'FOX'
# FOX = yf.download(symbol, start=start_date, end=end_date)
# FOX = FOX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'WBD'
# WBD = yf.download(symbol, start=start_date, end=end_date)
# WBD = WBD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'DIS'
# DIS = yf.download(symbol, start=start_date, end=end_date)
# DIS = DIS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'UPS'
# UPS = yf.download(symbol, start=start_date, end=end_date)
# UPS = UPS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'FDX'
# FDX = yf.download(symbol, start=start_date, end=end_date)
# FDX = FDX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'DAL'
# DAL = yf.download(symbol, start=start_date, end=end_date)
# DAL = DAL.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'AAL'
# AAL = yf.download(symbol, start=start_date, end=end_date)
# AAL = AAL.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'XOM'
# XOM = yf.download(symbol, start=start_date, end=end_date)
# XOM = XOM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CVX'
# CVX = yf.download(symbol, start=start_date, end=end_date)
# CVX = CVX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'BAC'
# BAC = yf.download(symbol, start=start_date, end=end_date)
# BAC = BAC.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'JPM'
# JPM = yf.download(symbol, start=start_date, end=end_date)
# JPM = JPM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MA'
# MA = yf.download(symbol, start=start_date, end=end_date)
# MA = MA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'V'
# V = yf.download(symbol, start=start_date, end=end_date)
# V = V.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SPG'
# SPG = yf.download(symbol, start=start_date, end=end_date)
# SPG = SPG.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'VNO'
# VNO = yf.download(symbol, start=start_date, end=end_date)
# VNO = VNO.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)

# symbol = 'MMM'
# MMM = yf.download(symbol, start=start_date, end=end_date)
# MMM = MMM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GE'
# GE = yf.download(symbol, start=start_date, end=end_date)
# GE = GE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'F'
# F = yf.download(symbol, start=start_date, end=end_date)
# F = F.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GM'
# GM = yf.download(symbol, start=start_date, end=end_date)
# GM = GM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HON'
# HON = yf.download(symbol, start=start_date, end=end_date)
# HON = HON.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'LMT'
# LMT = yf.download(symbol, start=start_date, end=end_date)
# LMT = LMT.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)


# # Futures



# symbol = 'ES=F'
# ESF = yf.download(symbol, start=start_date, end=end_date)
# ESF = ESF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'YM=F'
# YMF = yf.download(symbol, start=start_date, end=end_date)
# YMF = YMF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'NQ=F'
# NQF = yf.download(symbol, start=start_date, end=end_date)
# NQF = NQF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'RTY=F'
# RTYF = yf.download(symbol, start=start_date, end=end_date)
# RTYF = RTYF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZB=F'
# ZBF = yf.download(symbol, start=start_date, end=end_date)
# ZBF = ZBF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZN=F'
# ZNF = yf.download(symbol, start=start_date, end=end_date)
# ZNF = ZNF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZF=F'
# ZFF = yf.download(symbol, start=start_date, end=end_date)
# ZFF = ZFF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZT=F'
# ZTF = yf.download(symbol, start=start_date, end=end_date)
# ZTF = ZTF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GC=F'
# GCF = yf.download(symbol, start=start_date, end=end_date)
# GCF = GCF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HG=F'
# HGF = yf.download(symbol, start=start_date, end=end_date)
# HGF = HGF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SI=F'
# SIF = yf.download(symbol, start=start_date, end=end_date)
# SIF = SIF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'PL=F'
# PLF = yf.download(symbol, start=start_date, end=end_date)
# PLF = PLF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# CLF = yf.download(symbol, start=start_date, end=end_date)
# CLF = CLF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'NG=F'
# NGF = yf.download(symbol, start=start_date, end=end_date)
# NGF = NGF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'BZ=F'
# BZF = yf.download(symbol, start=start_date, end=end_date)
# BZF = BZF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZC=F'
# ZCF = yf.download(symbol, start=start_date, end=end_date)
# ZCF = ZCF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZO=F'
# ZOF = yf.download(symbol, start=start_date, end=end_date)
# ZOF = ZOF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'KE=F'
# KEF = yf.download(symbol, start=start_date, end=end_date)
# KEF = KEF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZR=F'
# ZRF = yf.download(symbol, start=start_date, end=end_date)
# ZRF = ZRF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZM=F'
# ZMF = yf.download(symbol, start=start_date, end=end_date)
# ZMF = ZMF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZL=F'
# ZLF = yf.download(symbol, start=start_date, end=end_date)
# ZLF = ZLF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ZS=F'
# ZSF = yf.download(symbol, start=start_date, end=end_date)
# ZSF = ZSF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'GF=F'
# GFF = yf.download(symbol, start=start_date, end=end_date)
# GFF = GFF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HE=F'
# HEF = yf.download(symbol, start=start_date, end=end_date)
# HEF = HEF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HO=F'
# HOF = yf.download(symbol, start=start_date, end=end_date)
# HOF = HOF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'LE=F'
# LFF = yf.download(symbol, start=start_date, end=end_date)
# LFF = LFF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CC=F'
# CCF = yf.download(symbol, start=start_date, end=end_date)
# CCF = CCF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'KC=F'
# KCF = yf.download(symbol, start=start_date, end=end_date)
# KCF = KCF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CT=F'
# CTF = yf.download(symbol, start=start_date, end=end_date)
# CTF = CTF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'OJ=F'
# OJF = yf.download(symbol, start=start_date, end=end_date)
# OJF = OJF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SB=F'
# SBF = yf.download(symbol, start=start_date, end=end_date)
# SBF = SBF.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)


# # EFTs

# symbol = 'KBA'
# KBA = yf.download(symbol, start=start_date, end=end_date)
# KBA = KBA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CHIQ'
# CHIQ = yf.download(symbol, start=start_date, end=end_date)
# CHIQ = CHIQ.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CNTX'
# CNTX = yf.download(symbol, start=start_date, end=end_date)
# CNTX = CNTX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CHIS'
# CHIS = yf.download(symbol, start=start_date, end=end_date)
# CHIS = CHIS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CNYA'
# CNYA = yf.download(symbol, start=start_date, end=end_date)
# CNYA = CNYA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'ASHX'
# ASHX = yf.download(symbol, start=start_date, end=end_date)
# ASHX = ASHX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'KFYP'
# KFYP = yf.download(symbol, start=start_date, end=end_date)
# KFYP = KFYP.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'KGRN'
# KGRN = yf.download(symbol, start=start_date, end=end_date)
# KGRN = KGRN.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'THD'
# THD = yf.download(symbol, start=start_date, end=end_date)
# THD = THD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'BBAX'
# BBAX = yf.download(symbol, start=start_date, end=end_date)
# BBAX = BBAX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'FEMS'
# FEMS = yf.download(symbol, start=start_date, end=end_date)
# FEMS = FEMS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'EZA'
# EZA = yf.download(symbol, start=start_date, end=end_date)
# EZA = EZA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'XSD'
# XSD = yf.download(symbol, start=start_date, end=end_date)
# XSD = XSD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'EYLD'
# EYLD = yf.download(symbol, start=start_date, end=end_date)
# EYLD = EYLD.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'FNDE'
# FNDE = yf.download(symbol, start=start_date, end=end_date)
# FNDE = FNDE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'SPEM'
# SPEM = yf.download(symbol, start=start_date, end=end_date)
# SPEM = SPEM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'DXJS'
# DXJS = yf.download(symbol, start=start_date, end=end_date)
# DXJS = DXJS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'KURE'
# KURE = yf.download(symbol, start=start_date, end=end_date)
# KURE = KURE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'EWX'
# EWX = yf.download(symbol, start=start_date, end=end_date)
# EWX = EWX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'FLJH'
# FLJH = yf.download(symbol, start=start_date, end=end_date)
# FLJH = FLJH.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CQQQ'
# CQQQ = yf.download(symbol, start=start_date, end=end_date)
# CQQQ = CQQQ.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'CHIE'
# CHIE = yf.download(symbol, start=start_date, end=end_date)
# CHIE = CHIE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'MFEM'
# MFEM = yf.download(symbol, start=start_date, end=end_date)
# MFEM = MFEM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'DGS'
# DGS = yf.download(symbol, start=start_date, end=end_date)
# DGS = DGS.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = 'HEEM'
# HEEM = yf.download(symbol, start=start_date, end=end_date)
# HEEM = HEEM.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)


# # World Composite Index


# symbol = '^HSI'
# HSI = yf.download(symbol, start=start_date, end=end_date)
# HSI = HSI.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '000001.SS'
# SSE = yf.download(symbol, start=start_date, end=end_date)
# SSE = SSE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^N225'
# N225 = yf.download(symbol, start=start_date, end=end_date)
# N225 = N225.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^KS11'
# KS11 = yf.download(symbol, start=start_date, end=end_date)
# KS11 = KS11.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^BSESN'
# BSESN = yf.download(symbol, start=start_date, end=end_date)
# BSESN = BSESN.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^MXX'
# MXX = yf.download(symbol, start=start_date, end=end_date)
# MXX = MXX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^TNX'
# TNX = yf.download(symbol, start=start_date, end=end_date)
# TNX = TNX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^VIX'
# VIX = yf.download(symbol, start=start_date, end=end_date)
# VIX = VIX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^BVSP'
# BVSP = yf.download(symbol, start=start_date, end=end_date)
# BVSP = BVSP.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^IXIC'
# IXIC = yf.download(symbol, start=start_date, end=end_date)
# IXIC = IXIC.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^GSPTSE'
# GSPTSE = yf.download(symbol, start=start_date, end=end_date)
# GSPTSE = GSPTSE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^DJI'
# DJI = yf.download(symbol, start=start_date, end=end_date)
# DJI = DJI.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^FCHI'
# FCHI = yf.download(symbol, start=start_date, end=end_date)
# FCHI = FCHI.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^GDAXI'
# GDAXI = yf.download(symbol, start=start_date, end=end_date)
# GDAXI = GDAXI.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^FTSE'
# FTSE = yf.download(symbol, start=start_date, end=end_date)
# FTSE = FTSE.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^IBEX'
# IBEX = yf.download(symbol, start=start_date, end=end_date)
# IBEX = IBEX.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^N100'
# N100 = yf.download(symbol, start=start_date, end=end_date)
# N100 = N100.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^GSPC'
# GSPC = yf.download(symbol, start=start_date, end=end_date)
# GSPC = GSPC.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^RUT'
# RUT = yf.download(symbol, start=start_date, end=end_date)
# RUT = RUT.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^NYA'
# NYA = yf.download(symbol, start=start_date, end=end_date)
# NYA = NYA.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^STI'
# STI = yf.download(symbol, start=start_date, end=end_date)
# STI = STI.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)
# symbol = '^AXJO'
# AXJO = yf.download(symbol, start=start_date, end=end_date)
# AXJO = AXJO.drop(['Adj Close', 'Volume', 'Low', 'High'], axis=1)



In [ ]:
# # Chip Maker

# import yfinance as yf

# start_date = '2013-01-01'
# end_date = '2023-09-26'
# symbol = 'TSM'
# TSM = yf.download(symbol, start=start_date, end=end_date)
# TSM = TSM.drop('Adj Close', axis=1)
# symbol = 'INTC'
# INTC = yf.download(symbol, start=start_date, end=end_date)
# INTC = INTC.drop('Adj Close', axis=1)
# symbol = 'ASML'
# ASML = yf.download(symbol, start=start_date, end=end_date)
# ASML = ASML.drop('Adj Close', axis=1)
# symbol = 'MU'
# MU = yf.download(symbol, start=start_date, end=end_date)
# MU = MU.drop('Adj Close', axis=1)
# symbol = 'NVDA'
# NVDA = yf.download(symbol, start=start_date, end=end_date)
# NVDA = NVDA.drop('Adj Close', axis=1)
# symbol = 'AMD'
# AMD = yf.download(symbol, start=start_date, end=end_date)
# AMD = AMD.drop('Adj Close', axis=1)
# symbol = 'QCOM'
# QCOM = yf.download(symbol, start=start_date, end=end_date)
# QCOM = QCOM.drop('Adj Close', axis=1)
# symbol = 'SNPS'
# SNPS = yf.download(symbol, start=start_date, end=end_date)
# SNPS = SNPS.drop('Adj Close', axis=1)
# symbol = 'MRVL'
# MRVL = yf.download(symbol, start=start_date, end=end_date)
# MRVL = MRVL.drop('Adj Close', axis=1)
# symbol = '^IXIC'
# IXIC = yf.download(symbol, start=start_date, end=end_date)
# IXIC = IXIC.drop('Adj Close', axis=1)

# # information technology

# symbol = 'AAPL'
# AAPL = yf.download(symbol, start=start_date, end=end_date)
# AAPL = AAPL.drop('Adj Close', axis=1)
# symbol = 'MSFT'
# MSFT = yf.download(symbol, start=start_date, end=end_date)
# MSFT = MSFT.drop('Adj Close', axis=1)
# symbol = 'TSLA'
# TSLA = yf.download(symbol, start=start_date, end=end_date)
# TSLA = TSLA.drop('Adj Close', axis=1)
# symbol = 'GOOGL'
# GOOGL = yf.download(symbol, start=start_date, end=end_date)
# GOOGL = GOOGL.drop('Adj Close', axis=1)
# symbol = 'GOOG'
# GOOG = yf.download(symbol, start=start_date, end=end_date)
# GOOG = GOOG.drop('Adj Close', axis=1)
# symbol = 'AMZN'
# AMZN = yf.download(symbol, start=start_date, end=end_date)
# AMZN = AMZN.drop('Adj Close', axis=1)
# symbol = 'META'
# META = yf.download(symbol, start=start_date, end=end_date)
# META = META.drop('Adj Close', axis=1)
# symbol = 'AMD'
# AMD = yf.download(symbol, start=start_date, end=end_date)
# AMD = AMD.drop('Adj Close', axis=1)
# symbol = 'ASML'
# ASML = yf.download(symbol, start=start_date, end=end_date)
# ASML = ASML.drop('Adj Close', axis=1)
# symbol = 'NVDA'
# NVDA = yf.download(symbol, start=start_date, end=end_date)
# NVDA = NVDA.drop('Adj Close', axis=1)
# symbol = 'IBM'
# IBM = yf.download(symbol, start=start_date, end=end_date)
# IBM = IBM.drop('Adj Close', axis=1)
# symbol = 'NFLX'
# NFLX = yf.download(symbol, start=start_date, end=end_date)
# NFLX = NFLX.drop('Adj Close', axis=1)

# # Consumer


# symbol = 'WMT'
# WMT = yf.download(symbol, start=start_date, end=end_date)
# WMT = WMT.drop('Adj Close', axis=1)
# symbol = 'TGT'
# TGT = yf.download(symbol, start=start_date, end=end_date)
# TGT = TGT.drop('Adj Close', axis=1)
# symbol = 'COST'
# COST = yf.download(symbol, start=start_date, end=end_date)
# COST = COST.drop('Adj Close', axis=1)
# symbol = 'HD'
# HD = yf.download(symbol, start=start_date, end=end_date)
# HD = HD.drop('Adj Close', axis=1)
# symbol = 'LOW'
# LOW = yf.download(symbol, start=start_date, end=end_date)
# LOW = LOW.drop('Adj Close', axis=1)

# symbol = 'PG'
# PG = yf.download(symbol, start=start_date, end=end_date)
# PG = PG.drop('Adj Close', axis=1)
# symbol = 'JNJ'
# JNJ = yf.download(symbol, start=start_date, end=end_date)
# JNJ = JNJ.drop('Adj Close', axis=1)
# symbol = 'PFE'
# PFE = yf.download(symbol, start=start_date, end=end_date)
# PFE = PFE.drop('Adj Close', axis=1)
# symbol = 'CVS'
# CVS = yf.download(symbol, start=start_date, end=end_date)
# CVS = CVS.drop('Adj Close', axis=1)

# symbol = 'KO'
# KO = yf.download(symbol, start=start_date, end=end_date)
# KO = KO.drop('Adj Close', axis=1)
# symbol = 'PEP'
# PEP = yf.download(symbol, start=start_date, end=end_date)
# PEP = PEP.drop('Adj Close', axis=1)

# symbol = 'NKE'
# NKE = yf.download(symbol, start=start_date, end=end_date)
# NKE = NKE.drop('Adj Close', axis=1)
# symbol = 'MCD'
# MCD = yf.download(symbol, start=start_date, end=end_date)
# MCD = MCD.drop('Adj Close', axis=1)
# symbol = 'SBUX'
# SBUX = yf.download(symbol, start=start_date, end=end_date)
# SBUX = SBUX.drop('Adj Close', axis=1)

# symbol = 'VZ'
# VZ = yf.download(symbol, start=start_date, end=end_date)
# VZ = VZ.drop('Adj Close', axis=1)
# symbol = 'T'
# T = yf.download(symbol, start=start_date, end=end_date)
# T = T.drop('Adj Close', axis=1)
# symbol = 'FOX'
# FOX = yf.download(symbol, start=start_date, end=end_date)
# FOX = FOX.drop('Adj Close', axis=1)
# symbol = 'WBD'
# WBD = yf.download(symbol, start=start_date, end=end_date)
# WBD = WBD.drop('Adj Close', axis=1)
# symbol = 'DIS'
# DIS = yf.download(symbol, start=start_date, end=end_date)
# DIS = DIS.drop('Adj Close', axis=1)

# symbol = 'UPS'
# UPS = yf.download(symbol, start=start_date, end=end_date)
# UPS = UPS.drop('Adj Close', axis=1)
# symbol = 'FDX'
# FDX = yf.download(symbol, start=start_date, end=end_date)
# FDX = FDX.drop('Adj Close', axis=1)
# symbol = 'DAL'
# DAL = yf.download(symbol, start=start_date, end=end_date)
# DAL = DAL.drop('Adj Close', axis=1)
# symbol = 'AAL'
# AAL = yf.download(symbol, start=start_date, end=end_date)
# AAL = AAL.drop('Adj Close', axis=1)
# symbol = 'XOM'
# XOM = yf.download(symbol, start=start_date, end=end_date)
# XOM = XOM.drop('Adj Close', axis=1)
# symbol = 'CVX'
# CVX = yf.download(symbol, start=start_date, end=end_date)
# CVX = CVX.drop('Adj Close', axis=1)

# symbol = 'BAC'
# BAC = yf.download(symbol, start=start_date, end=end_date)
# BAC = BAC.drop('Adj Close', axis=1)
# symbol = 'JPM'
# JPM = yf.download(symbol, start=start_date, end=end_date)
# JPM = JPM.drop('Adj Close', axis=1)
# symbol = 'MA'
# MA = yf.download(symbol, start=start_date, end=end_date)
# MA = MA.drop('Adj Close', axis=1)
# symbol = 'V'
# V = yf.download(symbol, start=start_date, end=end_date)
# V = V.drop('Adj Close', axis=1)
# symbol = 'SPG'
# SPG = yf.download(symbol, start=start_date, end=end_date)
# SPG = SPG.drop('Adj Close', axis=1)
# symbol = 'VNO'
# VNO = yf.download(symbol, start=start_date, end=end_date)
# VNO = VNO.drop('Adj Close', axis=1)

# symbol = 'MMM'
# MMM = yf.download(symbol, start=start_date, end=end_date)
# MMM = MMM.drop('Adj Close', axis=1)
# symbol = 'GE'
# GE = yf.download(symbol, start=start_date, end=end_date)
# GE = GE.drop('Adj Close', axis=1)
# symbol = 'F'
# F = yf.download(symbol, start=start_date, end=end_date)
# F = F.drop('Adj Close', axis=1)
# symbol = 'GM'
# GM = yf.download(symbol, start=start_date, end=end_date)
# GM = GM.drop('Adj Close', axis=1)
# symbol = 'HON'
# HON = yf.download(symbol, start=start_date, end=end_date)
# HON = HON.drop('Adj Close', axis=1)
# symbol = 'LMT'
# LMT = yf.download(symbol, start=start_date, end=end_date)
# LMT = LMT.drop('Adj Close', axis=1)


# # Futures



# symbol = 'ES=F'
# ESF = yf.download(symbol, start=start_date, end=end_date)
# ESF = ESF.drop('Adj Close', axis=1)
# symbol = 'YM=F'
# YMF = yf.download(symbol, start=start_date, end=end_date)
# YMF = YMF.drop('Adj Close', axis=1)
# symbol = 'NQ=F'
# NQF = yf.download(symbol, start=start_date, end=end_date)
# NQF = NQF.drop('Adj Close', axis=1)
# symbol = 'RTY=F'
# RTYF = yf.download(symbol, start=start_date, end=end_date)
# RTYF = RTYF.drop('Adj Close', axis=1)
# symbol = 'ZB=F'
# ZBF = yf.download(symbol, start=start_date, end=end_date)
# ZBF = ZBF.drop('Adj Close', axis=1)
# symbol = 'ZN=F'
# ZNF = yf.download(symbol, start=start_date, end=end_date)
# ZNF = ZNF.drop('Adj Close', axis=1)
# symbol = 'ZF=F'
# ZFF = yf.download(symbol, start=start_date, end=end_date)
# ZFF = ZFF.drop('Adj Close', axis=1)
# symbol = 'ZT=F'
# ZTF = yf.download(symbol, start=start_date, end=end_date)
# ZTF = ZTF.drop('Adj Close', axis=1)
# symbol = 'GC=F'
# GCF = yf.download(symbol, start=start_date, end=end_date)
# GCF = GCF.drop('Adj Close', axis=1)
# symbol = 'HG=F'
# HGF = yf.download(symbol, start=start_date, end=end_date)
# HGF = HGF.drop('Adj Close', axis=1)
# symbol = 'SI=F'
# SIF = yf.download(symbol, start=start_date, end=end_date)
# SIF = SIF.drop('Adj Close', axis=1)
# symbol = 'PL=F'
# PLF = yf.download(symbol, start=start_date, end=end_date)
# PLF = PLF.drop('Adj Close', axis=1)
# CLF = yf.download(symbol, start=start_date, end=end_date)
# CLF = CLF.drop('Adj Close', axis=1)
# symbol = 'NG=F'
# NGF = yf.download(symbol, start=start_date, end=end_date)
# NGF = NGF.drop('Adj Close', axis=1)
# symbol = 'BZ=F'
# BZF = yf.download(symbol, start=start_date, end=end_date)
# BZF = BZF.drop('Adj Close', axis=1)
# symbol = 'ZC=F'
# ZCF = yf.download(symbol, start=start_date, end=end_date)
# ZCF = ZCF.drop('Adj Close', axis=1)
# symbol = 'ZO=F'
# ZOF = yf.download(symbol, start=start_date, end=end_date)
# ZOF = ZOF.drop('Adj Close', axis=1)
# symbol = 'KE=F'
# KEF = yf.download(symbol, start=start_date, end=end_date)
# KEF = KEF.drop('Adj Close', axis=1)
# symbol = 'ZR=F'
# ZRF = yf.download(symbol, start=start_date, end=end_date)
# ZRF = ZRF.drop('Adj Close', axis=1)
# symbol = 'ZM=F'
# ZMF = yf.download(symbol, start=start_date, end=end_date)
# ZMF = ZMF.drop('Adj Close', axis=1)
# symbol = 'ZL=F'
# ZLF = yf.download(symbol, start=start_date, end=end_date)
# ZLF = ZLF.drop('Adj Close', axis=1)
# symbol = 'ZS=F'
# ZSF = yf.download(symbol, start=start_date, end=end_date)
# ZSF = ZSF.drop('Adj Close', axis=1)
# symbol = 'GF=F'
# GFF = yf.download(symbol, start=start_date, end=end_date)
# GFF = GFF.drop('Adj Close', axis=1)
# symbol = 'HE=F'
# HEF = yf.download(symbol, start=start_date, end=end_date)
# HEF = HEF.drop('Adj Close', axis=1)
# symbol = 'HO=F'
# HOF = yf.download(symbol, start=start_date, end=end_date)
# HOF = HOF.drop('Adj Close', axis=1)
# symbol = 'LE=F'
# LFF = yf.download(symbol, start=start_date, end=end_date)
# LFF = LFF.drop('Adj Close', axis=1)
# symbol = 'CC=F'
# CCF = yf.download(symbol, start=start_date, end=end_date)
# CCF = CCF.drop('Adj Close', axis=1)
# symbol = 'KC=F'
# KCF = yf.download(symbol, start=start_date, end=end_date)
# KCF = KCF.drop('Adj Close', axis=1)
# symbol = 'CT=F'
# CTF = yf.download(symbol, start=start_date, end=end_date)
# CTF = CTF.drop('Adj Close', axis=1)
# symbol = 'OJ=F'
# OJF = yf.download(symbol, start=start_date, end=end_date)
# OJF = OJF.drop('Adj Close', axis=1)
# symbol = 'SB=F'
# SBF = yf.download(symbol, start=start_date, end=end_date)
# SBF = SBF.drop('Adj Close', axis=1)


# # EFTs

# symbol = 'KBA'
# KBA = yf.download(symbol, start=start_date, end=end_date)
# KBA = KBA.drop('Adj Close', axis=1)
# symbol = 'CHIQ'
# CHIQ = yf.download(symbol, start=start_date, end=end_date)
# CHIQ = CHIQ.drop('Adj Close', axis=1)
# symbol = 'CNTX'
# CNTX = yf.download(symbol, start=start_date, end=end_date)
# CNTX = CNTX.drop('Adj Close', axis=1)
# symbol = 'CHIS'
# CHIS = yf.download(symbol, start=start_date, end=end_date)
# CHIS = CHIS.drop('Adj Close', axis=1)
# symbol = 'CNYA'
# CNYA = yf.download(symbol, start=start_date, end=end_date)
# CNYA = CNYA.drop('Adj Close', axis=1)
# symbol = 'ASHX'
# ASHX = yf.download(symbol, start=start_date, end=end_date)
# ASHX = ASHX.drop('Adj Close', axis=1)
# symbol = 'KFYP'
# KFYP = yf.download(symbol, start=start_date, end=end_date)
# KFYP = KFYP.drop('Adj Close', axis=1)
# symbol = 'KGRN'
# KGRN = yf.download(symbol, start=start_date, end=end_date)
# KGRN = KGRN.drop('Adj Close', axis=1)
# symbol = 'THD'
# THD = yf.download(symbol, start=start_date, end=end_date)
# THD = THD.drop('Adj Close', axis=1)
# symbol = 'BBAX'
# BBAX = yf.download(symbol, start=start_date, end=end_date)
# BBAX = BBAX.drop('Adj Close', axis=1)
# symbol = 'FEMS'
# FEMS = yf.download(symbol, start=start_date, end=end_date)
# FEMS = FEMS.drop('Adj Close', axis=1)
# symbol = 'EZA'
# EZA = yf.download(symbol, start=start_date, end=end_date)
# EZA = EZA.drop('Adj Close', axis=1)
# symbol = 'XSD'
# XSD = yf.download(symbol, start=start_date, end=end_date)
# XSD = XSD.drop('Adj Close', axis=1)
# symbol = 'EYLD'
# EYLD = yf.download(symbol, start=start_date, end=end_date)
# EYLD = EYLD.drop('Adj Close', axis=1)
# symbol = 'FNDE'
# FNDE = yf.download(symbol, start=start_date, end=end_date)
# FNDE = FNDE.drop('Adj Close', axis=1)
# symbol = 'SPEM'
# SPEM = yf.download(symbol, start=start_date, end=end_date)
# SPEM = SPEM.drop('Adj Close', axis=1)
# symbol = 'DXJS'
# DXJS = yf.download(symbol, start=start_date, end=end_date)
# DXJS = DXJS.drop('Adj Close', axis=1)
# symbol = 'KURE'
# KURE = yf.download(symbol, start=start_date, end=end_date)
# KURE = KURE.drop('Adj Close', axis=1)
# symbol = 'EWX'
# EWX = yf.download(symbol, start=start_date, end=end_date)
# EWX = EWX.drop('Adj Close', axis=1)
# symbol = 'FLJH'
# FLJH = yf.download(symbol, start=start_date, end=end_date)
# FLJH = FLJH.drop('Adj Close', axis=1)
# symbol = 'CQQQ'
# CQQQ = yf.download(symbol, start=start_date, end=end_date)
# CQQQ = CQQQ.drop('Adj Close', axis=1)
# symbol = 'CHIE'
# CHIE = yf.download(symbol, start=start_date, end=end_date)
# CHIE = CHIE.drop('Adj Close', axis=1)
# symbol = 'MFEM'
# MFEM = yf.download(symbol, start=start_date, end=end_date)
# MFEM = MFEM.drop('Adj Close', axis=1)
# symbol = 'DGS'
# DGS = yf.download(symbol, start=start_date, end=end_date)
# DGS = DGS.drop('Adj Close', axis=1)
# symbol = 'HEEM'
# HEEM = yf.download(symbol, start=start_date, end=end_date)
# HEEM = HEEM.drop('Adj Close', axis=1)


# # World Composite Index


# symbol = '^HSI'
# HSI = yf.download(symbol, start=start_date, end=end_date)
# HSI = HSI.drop('Adj Close', axis=1)
# symbol = '000001.SS'
# SSE = yf.download(symbol, start=start_date, end=end_date)
# SSE = SSE.drop('Adj Close', axis=1)
# symbol = '^N225'
# N225 = yf.download(symbol, start=start_date, end=end_date)
# N225 = N225.drop('Adj Close', axis=1)
# symbol = '^KS11'
# KS11 = yf.download(symbol, start=start_date, end=end_date)
# KS11 = KS11.drop('Adj Close', axis=1)
# symbol = '^BSESN'
# BSESN = yf.download(symbol, start=start_date, end=end_date)
# BSESN = BSESN.drop('Adj Close', axis=1)
# symbol = '^MXX'
# MXX = yf.download(symbol, start=start_date, end=end_date)
# MXX = MXX.drop('Adj Close', axis=1)
# symbol = '^TNX'
# TNX = yf.download(symbol, start=start_date, end=end_date)
# TNX = TNX.drop('Adj Close', axis=1)
# symbol = '^VIX'
# VIX = yf.download(symbol, start=start_date, end=end_date)
# VIX = VIX.drop('Adj Close', axis=1)
# symbol = '^BVSP'
# BVSP = yf.download(symbol, start=start_date, end=end_date)
# BVSP = BVSP.drop('Adj Close', axis=1)
# symbol = '^IXIC'
# IXIC = yf.download(symbol, start=start_date, end=end_date)
# IXIC = IXIC.drop('Adj Close', axis=1)
# symbol = '^GSPTSE'
# GSPTSE = yf.download(symbol, start=start_date, end=end_date)
# GSPTSE = GSPTSE.drop('Adj Close', axis=1)
# symbol = '^DJI'
# DJI = yf.download(symbol, start=start_date, end=end_date)
# DJI = DJI.drop('Adj Close', axis=1)
# symbol = '^FCHI'
# FCHI = yf.download(symbol, start=start_date, end=end_date)
# FCHI = FCHI.drop('Adj Close', axis=1)
# symbol = '^GDAXI'
# GDAXI = yf.download(symbol, start=start_date, end=end_date)
# GDAXI = GDAXI.drop('Adj Close', axis=1)
# symbol = '^FTSE'
# FTSE = yf.download(symbol, start=start_date, end=end_date)
# FTSE = FTSE.drop('Adj Close', axis=1)
# symbol = '^IBEX'
# IBEX = yf.download(symbol, start=start_date, end=end_date)
# IBEX = IBEX.drop('Adj Close', axis=1)
# symbol = '^N100'
# N100 = yf.download(symbol, start=start_date, end=end_date)
# N100 = N100.drop('Adj Close', axis=1)
# symbol = '^GSPC'
# GSPC = yf.download(symbol, start=start_date, end=end_date)
# GSPC = GSPC.drop('Adj Close', axis=1)
# symbol = '^RUT'
# RUT = yf.download(symbol, start=start_date, end=end_date)
# RUT = RUT.drop('Adj Close', axis=1)
# symbol = '^NYA'
# NYA = yf.download(symbol, start=start_date, end=end_date)
# NYA = NYA.drop('Adj Close', axis=1)
# symbol = '^STI'
# STI = yf.download(symbol, start=start_date, end=end_date)
# STI = STI.drop('Adj Close', axis=1)
# symbol = '^AXJO'
# AXJO = yf.download(symbol, start=start_date, end=end_date)
# AXJO = AXJO.drop('Adj Close', axis=1)

In [ ]:
# test_inverse transformation
import csv
# train_data_list = [data3, data4, data5, data6, data7, data8, data9, data10
#                   , data11, data12, data13, data14, data15, data16, data17, data18, data19, data20]
# data21, data22, data23, data24, data25, data26, data27, data28, data29, data30, data31, data32, data33, data34, data35, data36, data37, data38, data39, data40
data = None
# train_data_list=[HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
                WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,
                 MMM,GE,F,GM,HON,LMT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,AMD,ASML,TSM,INTC,MU,QCOM,SNPS,MRVL,IBM,NFLX,MSFT,
                 SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO, 
                 ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,CLF,GCF,HGF,SIF,NGF,BZF,ZOF,KEF,ZRF,ZMF,ZLF,ZCF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                 WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,
#                  ]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                 WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,
#                  MMM,GE,F,GM,HON,LMT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,AMD,ASML,TSLA,TSM,INTC,MU,QCOM,SNPS,MRVL,IBM,NFLX,MSFT,
#                  SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                  ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,GCF,HGF,SIF,NGF,BZF,ZCF,ZOF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                  ]
# train_data_list=[ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,CLF,GCF,HGF,SIF,NGF,BZF,ZCF,ZOF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF]
# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,
#                  FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,GSPC,DJI,RUT,
#                  HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,ESF,YMF,NQF,
#                  RTYF,ZBF,ZNF,CLF,GCF,HGF,SIF,CLF,NGF,ZCF,KEF,MSFT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,TSLA,AMD,ASML]

# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FOX,
#                  FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,GSPC,DJI,PFE,CVS,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS]
# valid_data_list=[MSFT,AAPL,GOOGL,AMZN,META,TSLA,AMD,ASML,NVDA,TSM]
# valid_data_list=[ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,CLF,GCF,HGF,SIF,NGF,BZF,ZOF,KEF,ZRF,ZMF,ZLF,ZCF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF]

# train_data_list=[ESF,YMF,NQF,RTYF,ZBF,ZNF,CLF,GCF,HGF,SIF,CLF,NGF,ZCF,ZFF,ZTF,PLF,PAF,BZF,ZOF,KCF,CTF]
# train_data_list=[ESF,YMF,NQF,ZBF,ZNF,GCF,HGF,SIF,CLF,ZCF,NGF,WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,CVS,KO,PEP]

# valid_data_list = [SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO, 
#                  ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,CLF,GCF,HGF,SIF,NGF,BZF,ZOF,KEF,ZRF,ZMF,ZLF,ZCF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF]

valid_data_list=[TSLA]

# valid_data_list = [ESF]

symbol = 'TSLA'
start_date = '2022-01-01'
end_date = '2023-09-26'
start_date_short = '2023-01-01'
end_date_short = '2023-09-26'
target_col = 'Close'

n_trials = 1
n_top_models = 3
n_predict = 5
n_last_sequence = 100
forward = 0
# valid_size = 0.1

# save_directory = "/home/young78703/Data_Science_Project/model_save/LSTM_TimeSeries_Regression/random_search/tsla_new/tsla_1"
# results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df = random_search(
#     data=data, train_data_list=train_data_list, valid_data_list=valid_data_list, symbol=symbol, start_date=start_date, end_date=end_date, 
#     start_date_short=start_date_short, end_date_short=end_date_short, target_col=target_col, n_trials=n_trials, n_top_models=n_top_models, 
#     model_save=True, save_directory=save_directory, plot_loss=False, predict_plot=True, future_plot=False, overall_future_plot=True, 
#     use_target_col=True, future_predictions=None, n_predict=n_predict, n_last_sequence=n_last_sequence, forward=forward)
save_directory = "/home/young78703/Data_Science_Project/model_save/LSTM_TimeSeries_Regression/random_search/tsla_new/tsla_test"
results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df = random_search(
    data=data, train_data_list=train_data_list, valid_data_list=valid_data_list, symbol=symbol, start_date=start_date, end_date=end_date, 
    start_date_short=start_date_short, end_date_short=end_date_short, target_col=target_col, n_trials=n_trials, n_top_models=n_top_models, 
    model_save=True, save_directory=save_directory, plot_loss=False, predict_plot=True, future_plot=False, overall_future_plot=True, 
    use_target_col=True, future_predictions=None, n_last_sequence=n_last_sequence, forward=forward)

output_file_path = "/home/young78703/Data_Science_Project/model_save/LSTM_TimeSeries_Regression/random_search/tsla_new/tsla_test"

# Save results_df
results_df.to_csv(f'{output_file_path}_results.csv', index=True)

#Save top_models
top_models_df = pd.DataFrame(top_models, columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
top_models_df.to_csv(f'{output_file_path}_top_models.csv', index=True)

# Save the future_metrics DataFrame to a CSV file
all_future_metric_finals.to_csv(f'{output_file_path}_all_future_metrics.csv', index=True)
# Save future_predictions
all_future_predictions_df.to_csv(f'{output_file_path}_all_future_predictions.csv', index=True)
# Save the overall_future_metrics DataFrame to a CSV file
all_overall_future_metrics_df.to_csv(f'{output_file_path}_all_overall_future_metrics.csv', index=True)


In [ ]:
# test_inverse transformation
import csv
# train_data_list = [data3, data4, data5, data6, data7, data8, data9, data10
#                   , data11, data12, data13, data14, data15, data16, data17, data18, data19, data20]
# data21, data22, data23, data24, data25, data26, data27, data28, data29, data30, data31, data32, data33, data34, data35, data36, data37, data38, data39, data40
data = None
# train_data_list=[HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                 WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,
#                  MMM,GE,F,GM,HON,LMT,MSFT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,TSLA,AMD,ASML,
#                  SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,VIX,IXIC,GSPTSE,N100,STI,AXJO]
# train_data_list=[KBA,CHIQ,CNTX,CHIS,CNYA,ASHX,KFYP,KGRN,THD,BBAX,FEMS,EZA,XSD,EYLD,FNDE,SPEM,DXJS,KURE,EWX,FLJH,CQQQ,CHIE,MFEM,DGS,HEEM,
#                  ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,GCF,HGF,SIF,NGF,BZF,ZCF,ZOF,ZRF,ZMF,ZLF,ZSF,GFF,HEF,HOF,LFF,CCF,KCF,CTF,OJF,SBF,
#                  ]
train_data_list=[ESF,NQF,RTYF,ZBF,ZNF,ZFF,ZTF,CLF,GCF]
# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,
#                  FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,GSPC,DJI,RUT,
#                  HSI,SSE,N225,KS11,BSESN,FCHI,GDAXI,FTSE,IBEX,GSPC,DJI,RUT,NYA,ESF,YMF,NQF,
#                  RTYF,ZBF,ZNF,CLF,GCF,HGF,SIF,CLF,NGF,ZCF,KEF,MSFT,AAPL,GOOGL,GOOG,NVDA,AMZN,META,TSLA,AMD,ASML]

# train_data_list=[WMT,TGT,COST,HD,LOW,PG,JNJ,PFE,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS,FOX,
#                  FDX,DAL,AAL,XOM,CVX,BAC,JPM,MA,V,SPG,VNO,MMM,GE,F,GM,HON,LMT,GSPC,DJI,PFE,CVS,KO,PEP,NKE,MCD,SBUX,VZ,T,FOX,WBD,DIS,UPS]
# valid_data_list=[MSFT,AAPL,GOOGL,AMZN,META,TSLA,AMD,ASML,NVDA,TSM]
# valid_data_list=[GOOG]

# train_data_list=[ESF,YMF,NQF,CLF,GCF]

valid_data_list = [ZCF]

symbol='ZC=F'
start_date = '2021-01-01'
end_date = '2023-09-26'
start_date_short = '2023-01-01'
end_date_short = '2023-09-26'
target_col = 'Close'


n_trials = 50
n_top_models = 1
# n_predict = 5
n_last_sequence = 100
forward = 0
# valid_size = 0.1
# save_directory = "/home/young78703/Data_Science_Project/model_save/Future_Stock_Price/rut_10_30"
# save_directory = "/home/young78703/Data_Science_Project/model_save/Fixed_Model/model1/shopify_50"
# save_directory = "/home/young78703/Data_Science_Project/model_save/Fixed_Model/shopify_50"
save_directory = "/home/young78703/Data_Science_Project/model_save/LSTM_TimeSeries_Regression/random_search/indicators/zc=f_test"
results_df, top_models, all_future_predictions_df, all_future_metric_finals, all_overall_future_metrics_df = random_search(
    data=data, train_data_list=train_data_list, valid_data_list=valid_data_list, symbol=symbol, start_date=start_date, end_date=end_date, 
    start_date_short=start_date_short, end_date_short=end_date_short, target_col=target_col, n_trials=n_trials, n_top_models=n_top_models, 
    model_save=True, save_directory=save_directory, plot_loss=False, predict_plot=True, future_plot=False, overall_future_plot=True, 
    use_target_col=False, future_predictions=None, n_last_sequence=n_last_sequence, forward=forward)

# output_file_path = "/home/young78703/Data_Science_Project/output/Fixed_Model/model1/shopify_50.csv"
# output_file_path = "/home/young78703/Data_Science_Project/output/Future_Stock_Price/rut/rut_50.csv"
output_file_path = "/home/young78703/Data_Science_Project/model_save/LSTM_TimeSeries_Regression/random_search/indicators/zc=f_test"

# Save results_df
results_df.to_csv(f'{output_file_path}_results.csv', index=True)

#Save top_models
top_models_df = pd.DataFrame(top_models, columns=["Trial", "Parameters", "Train MSE", "Train MAE", "Train R2", "Test MSE", "Test MAE", "Test R2"])
top_models_df.to_csv(f'{output_file_path}_top_models.csv', index=True)

# Save the future_metrics DataFrame to a CSV file
all_future_metric_finals.to_csv(f'{output_file_path}_all_future_metrics.csv', index=True)
# Save future_predictions
all_future_predictions_df.to_csv(f'{output_file_path}_all_future_predictions.csv', index=True)
# Save the overall_future_metrics DataFrame to a CSV file
all_overall_future_metrics_df.to_csv(f'{output_file_path}_all_overall_future_metrics.csv', index=True)


